# Predicting the Future: Classical → Deep → Foundation

*Three Generations of Machine Learning Applied to Quantitative Trading*

Built and backtested on [QuantConnect](https://www.quantconnect.com/).

Based on exercises from *Hands on AI Trading: with Python, QuantConnect, and AWS*.

---

**What this notebook contains:**
- Narrative walkthrough of three ML trading techniques
- Complete algorithm source code (QuantConnect Python)
- Interactive performance charts (Plotly)
- Key references and sources

**Requirements:** `pip install plotly`

## Predicting the Future: Classical → Deep → Foundation

### Slide 1 — Quantitative Trading: A First Look

Predicting the Future: Classical → Deep → Foundation

Three Generations of Machine Learning Applied to Quantitative Trading Built and Backtested on:

QuantConnect...

---

Thank you for joining us this evening. Before we begin, I'd like to say that this presenatation is intended for educational purposes ONLY! Nothing said here today is intended as financial advice. There is a likliehood you will lose money if any of these techniques are used incorrectly. So now, let's begin... Today I'm going to walk you through 3 generations of machine learning — from a simple linear model, to a neural network, to a pre-trained AI model — and we're going to ask one question: do the results actually improve? As a side note: the exercises referenced in this presentation are from _Hands on AI Trading: with Python, QuantConnect, and AWS_, which is publicly availble for purchase


### Slide 2 — The Question

Can machine learning predict financial markets?

We'll try three approaches, each more sophisticated than the last:

1. Ridge Regression — A classical linear model (1970)
2. Temporal CNN — A deep neural network (1989)
3. Amazon Chronos — A pre-trained foundation model (2024)

Same platform. Same time period. Increasing complexity.

---

Can machine learning predict financial markets? Today we're going to let the data speak. We have three strategies, each representing a different generation of ML, all backtested on QuantConnect over roughly the same 5-year window — January 2019 through April 2024. Same platform, same rules, same scrutiny. Let's see what happens.


**References:**

- Pik, J. et al. (2025). *Hands-On AI Trading with Python, QuantConnect, and AWS*. Wiley. — Primary source for all exercises.
- Hoerl, A. E. & Kennard, R. W. (1970). "Ridge Regression: Biased Estimation for Nonorthogonal Problems." *Technometrics*, 12(1), 55–67. — Ridge regression origin (1970).
- LeCun, Y. et al. (1989). "Backpropagation Applied to Handwritten Zip Code Recognition." *Neural Computation*, 1(4), 541–551. — CNN origin (1989).
- Ansari, A. F. et al. (2024). "Chronos: Learning the Language of Time Series." [arXiv:2403.07815](https://arxiv.org/abs/2403.07815). — Chronos foundation model (2024).

## Technique 1 — Ridge Regression (Classical ML)

### Slide 3 — Technique #1: Classical ML

Technique #1 — The Baseline

Ridge Regression: Predicting Futures Volatility

"Start simple. See if a linear model can add value."

---

Technique 1. We're going to start with the simplest possible approach: a linear regression model with a twist — ridge regularization. Why linear regression? In finance, interpretability matters — you want to know *which* features are driving your predictions and by how much, and a linear model gives you that directly through its coefficients. The problem is, when your input features are correlated — and in finance they almost always are — plain linear regression becomes unstable: small changes in the data can swing your coefficients wildly. That's where ridge comes in. It adds a penalty that shrinks all coefficients toward zero, which stabilizes the model without throwing any features away. The strategy is straightforward: predict how volatile each asset will be next week, then invest more in the less volatile ones, specifically *opening price* volatility. Inverse volatility weighting — a classic risk-parity adjacent technique. The idea behind risk parity is that every asset in your portfolio should contribute the same amount of *risk*, not the same amount of *capital*. We approximate that here: if an asset is expected to be twice as volatile, it gets half the weight. You're equalizing risk exposure across the portfolio rather than just spreading dollars evenly.


### Slide 4 — Technique #1: The Strategy

Strategy: Inverse Volatility Futures

Universe: 12 futures contracts across 3 sectors
- Indices: VIX, S&P 500, Nasdaq 100, DOW 30 (E-Mini)
- Energy: Brent Crude, Gasoline, Heating Oil, Natural Gas
- Grains: Corn, Oats, Soybeans, Wheat

Process:
1. Compute daily features: closing volatility, ATR, open interest
2. Train Ridge regression per contract (365-day rolling window)
3. Predict next-week volatility
4. Allocate inversely: less volatile → higher weight

Rebalancing: Weekly (Monday open)

---

We're trading 12 futures contracts — a diversified mix of stock indices, energy, and grains. Why these three sectors? Diversification. Stock indices, energy, and grains don't move in lockstep — they respond to different economic forces. That matters because our strategy is betting on _relative_ volatility: we want some contracts to be calm while others aren't, so the model has something to work with.

Every Monday, we look at three features for each contract. First, closing price volatility — how much has the closing price been bouncing around over the last three months? Second, average true range, or ATR — this captures intraday swings, not just close-to-close. It tells you how wild the ride is _within_ each day. Third, open interest — how many contracts are actively held. Rising open interest often signals that new money is flowing in, which can mean a trend is building. Together, these three give the model a picture of each contract's recent behavior from different angles.

We feed those into a ridge regression model — one model per contract — and it predicts how volatile the opening price will be over the next week. The model retrains every Monday on a rolling 365-day window. Why rolling? Markets change. A year of data is enough to learn stable patterns without letting stale data from two or three years ago dilute what's happening now.

Then we flip that prediction upside down: the contracts we expect to be _least_ volatile get the _most_ capital. It's risk management by prediction.


### Slide 5 — Technique #1: Ridge Regression

Ridge Regression — How It Works

Standard linear regression + an L2 penalty:

$J(θ) = MSE(θ) + (α/m) Σᵢ θᵢ²$

Why ridge? Our three features — volatility, ATR, open interest — are correlated. Ridge handles multicollinearity gracefully by shrinking all coefficients toward zero proportionally, without dropping any.

Inverse volatility weighting:

$wᵢ = C / σᵢ / Σⱼ(1/σⱼ) / multiplierᵢ$

where σᵢ = predicted volatility, C = 3 (position scaling)

---

Let me walk you through both equations on this slide. The first one is the cost function — J of theta — and it's what the model is trying to minimize. There are two terms. On the left, MSE of theta — mean squared error. That's the standard fitting term: how far off are the model's predictions from the actual values? On its own, minimizing MSE gives you ordinary least squares. The second term is the penalty: alpha over m, times the sum of each coefficient squared. That's the L2 regularization. It penalizes the model for having large coefficients. Alpha is a dial — when it's zero, you get plain linear regression. As you crank it up, the model cares more about keeping coefficients small and less about fitting the training data perfectly.

Why do we need that penalty here? Because our three features — closing volatility, ATR, and open interest — are correlated. ATR and closing volatility are both measuring price variability, just from slightly different angles. When you have correlated inputs, ordinary linear regression becomes unstable: small changes in the training data can swing coefficients wildly, even flip their signs. Ridge prevents that by shrinking all coefficients toward zero proportionally. It doesn't drop any feature — it just keeps every coefficient on a shorter leash. Think of it as OLS with a constraint: fit the data, but don't let any coefficient get too large.

The second equation is the allocation formula — how we turn the model's volatility predictions into actual portfolio weights. The numerator, C, is just a scaling constant — in our case, 3. The denominator has three parts. First, sigma-i: the predicted volatility for contract i. If an asset is expected to be more volatile, it gets a _smaller_ weight — that's the inverse relationship. Second, the sum of one over sigma-j across all contracts — that's a normalization factor. It ensures the weights add up to a consistent total, so we're not accidentally doubling our exposure when all assets happen to be calm. Third, the multiplier for contract i. This is crucial for futures: a single S&P E-Mini contract controls about $250,000 worth of the index, while a corn contract controls maybe $25,000 worth of grain. Without dividing by the multiplier, an equal _weight_ would mean wildly different dollar exposures. The multiplier converts from mathematical weight to economically equivalent position size.


### Slide 6 — Technique #1: Results

Results: Ridge Regression on Futures

Period: 2018-12-31 to 2024-04-01 · Starting capital: $100M

- Sharpe Ratio: 0.212
- CAGR: 5.85%
- Net Profit: +34.8%
- Alpha: -0.062
- Beta: 1.146
- Max Drawdown: 54.7%

Verdict: The strategy tracks the market with extra volatility. Alpha is negative. The model adds no value.

---

Here are the results. And let me walk you through what these numbers mean, because each one tells part of the story. Start with the Sharpe ratio: 0.212. The Sharpe ratio is return divided by risk — specifically, your excess return over the risk-free rate, divided by the standard deviation of those returns. A Sharpe of 1.0 is generally considered the threshold for _good_ in practice — it means you're getting one unit of return for every unit of risk you take on. 2.0 is excellent. We're at 0.212. For every unit of volatility this portfolio endures, we're earning about a fifth of a unit of return. That's barely worth the trouble.

Net profit of 34.8% over five and a quarter years — compound annual growth rate of 5.85%. That _sounds_ acceptable until you put it next to the market. Over roughly the same window — January 2019 through early 2024 — the S&P 500 more than doubled — total return of roughly 130%. Our strategy, which was taking _more_ risk than the market, returned about a quarter of what a passive index fund would have given you.

Now, alpha and beta — these tell the real story. Beta measures how much your strategy moves with the market. A beta of 1.0 means you move in lockstep with the benchmark. Ours is 1.146 — we're _amplifying_ market moves by about 15%. When the market rises 10%, we tend to rise about 11.5%. When it falls 10%, we fall 11.5%. That's not skill — that's effectively leveraged market exposure. Alpha is what's left after you account for that market exposure — the excess return the model generates on its own, independent of the market going up or down. Our alpha is negative 0.062. _Negative._ The model isn't finding any signal the market doesn't already give us. After you strip out the returns we'd get from beta alone, the strategy is _losing_ money.

And the maximum drawdown: 54.7%. At one point, more than half the portfolio value was gone from its peak. If you're managing a hundred million dollars and you have to explain that fifty-five million of it evaporated — but the model is working as designed — you won't have that client for long. That's not a risk-managed strategy. That's market exposure wearing a quantitative costume.

You can see it in the equity curve — the shape tracks the broad market. The peaks and troughs line up with market cycles, not with any independent signal. This is what happens when a linear model tries to predict a nonlinear system: it finds the strongest pattern available, which is just _the market going up over time_, and rides it. The ridge regression didn't learn anything about relative volatility that the market wasn't already pricing in.

Takeaway: a linear model on three correlated volatility features doesn't generate alpha. It just tracks the market with extra drawdown. So — what if we add complexity? What if instead of a linear model, we use a neural network that can learn nonlinear patterns across multiple time scales?


In [ ]:
# EX11 — main.py
# QuantConnect Algorithm Source
# This code runs on the QuantConnect platform
# (requires AlgorithmImports and QC runtime)

# region imports
from AlgorithmImports import *

from sklearn.linear_model import Ridge
# endregion


class InverseVolatilityRankAlgorithm(QCAlgorithm):
    """
    This algorithm demonstrates a way to use machine learning to form a 
    portfolio of Futures contracts where the weight of each contract is 
    the inverse of its expected future volatility. To forecast the
    future volatility, this strategy uses a ridge regression model and 
    the following factors: 
    - Volatility: Standard deviation of daily returns over the last 
    60 trading days
    - ATR: Average True Range over the last 60 trading days
    - Open interest
    """

    def initialize(self):
        self.set_start_date(2018, 12, 31)
        self.set_end_date(2024, 4, 1)
        self.set_cash(100_000_000)

        self._std_period = self.get_parameter('std_months', 3) * 26
        self._atr_period = self.get_parameter('atr_months', 3) * 26
        self._training_set_duration = timedelta(
            self.get_parameter('training_set_duration', 365)
        )
        self._future_std_period = 6

        self._contracts = []
        tickers = [
            Futures.Indices.VIX, 
            Futures.Indices.SP_500_E_MINI,
            Futures.Indices.NASDAQ_100_E_MINI,
            Futures.Indices.DOW_30_E_MINI,
            Futures.Energy.BRENT_CRUDE,
            Futures.Energy.GASOLINE,
            Futures.Energy.HEATING_OIL,
            Futures.Energy.NATURAL_GAS,
            Futures.Grains.CORN,
            Futures.Grains.OATS,
            Futures.Grains.SOYBEANS,
            Futures.Grains.WHEAT
        ]
        for ticker in tickers:
            future = self.add_future(ticker, extended_market_hours=True)
            future.set_filter(lambda universe: universe.front_month())
        
        schedule_symbol = Symbol.create("SPY", SecurityType.EQUITY, Market.USA)
        self.schedule.on(
            self.date_rules.week_start(schedule_symbol),
            self.time_rules.after_market_open(schedule_symbol, 1), 
            self._trade
        )

    def _trade(self):
        # Get the open interest factors.
        open_interest = self.history(
            OpenInterest, [c.symbol for c in self._contracts], 
            self._training_set_duration, fill_forward=False
        )
        open_interest.index = open_interest.index.droplevel(0)

        # Predict volatility over the next week for each security.
        expected_volatility_by_security = {}
        for security in self._contracts:
            symbol = security.symbol
            if symbol not in open_interest.index:
                continue
            # Get the factors.
            factors = pd.concat(
                [security.indicator_history, open_interest.loc[symbol]], 
                axis=1
            ).ffill().loc[security.indicator_history.index].dropna()
            if factors.empty:
                # The df can be empty if there is no open interest data 
                # for the asset (example: 
                # https://www.quantconnect.com/datasets/issue/16604).
                continue 
            # Get the labels.
            label = security.label_history

            # Align the factors and labels.
            idx = sorted(
                list(set(factors.index).intersection(set(label.index)))
            )
            # Ensure there are enough training samples.
            if len(idx) < 20:
                continue

            # Train the model.
            # Convert to numpy to avoid sklearn feature name type
            # mismatch (open_interest returns PandasColumn type,
            # indicator_history uses str).
            model = Ridge()
            model.fit(factors.loc[idx].values, label.loc[idx].values)

            # Predict the volatility over the next week.
            prediction = model.predict([factors.iloc[-1].values])[0] 
            if prediction > 0:
                expected_volatility_by_security[security] = prediction
                self.plot("Predictions", security.symbol.canonical.value, prediction)

        # Calculate the portfolio weights and rebalance.
        portfolio_targets = []
        std_sum = sum(
            [
                1 / expected_vol 
                for expected_vol in expected_volatility_by_security.values()
            ]
        )
        for security, expected_vol in expected_volatility_by_security.items():
            weight = (
                3 
                / expected_vol 
                / std_sum 
                / security.symbol_properties.contract_multiplier
            )
            # The numerator `3` above scales the position size.
            # If it's set to 1, the algorithm only trades a few of 
            # the Futures in the universe because of the
            # minimum_order_margin_portfolio_percentage setting. 
            # If it's set too high, we run into margin calls. 
            # 3 is the middle-ground where the algorithm trades 
            # most of the Futures without having to set 
            # minimum_order_margin_portfolio_percentage to zero.
            portfolio_targets.append(PortfolioTarget(security.symbol, weight))
        self.set_holdings(portfolio_targets, True)

    def on_securities_changed(self, changes):
        for security in changes.added_securities:
            if security.symbol.is_canonical(): 
                continue # Skip over continuous contracts.
            # Create the indicators.
            # We're using manual indicators here for demonstration
            # purposes. You can reduce the amount of code by using
            # the automatic indicators, but the following code
            # is an example of doing everything manually so you 
            # can have maximum control over the outcome.
            security.close_roc = RateOfChange(1)
            security.std_of_close_returns = IndicatorExtensions.of(
                StandardDeviation(self._std_period), 
                security.close_roc
            )
            security.atr = AverageTrueRange(self._atr_period)
            security.open_roc = RateOfChange(1)
            security.std_of_open_returns = IndicatorExtensions.of(
                StandardDeviation(self._future_std_period), security.open_roc
            )

            # Create some pandas objects to store the historical 
            # indicator values we'll use to train the ML.
            security.indicator_history = pd.DataFrame()
            security.label_history = pd.Series()

            # Create a consolidator to aggregate minute data into daily 
            # data for the indicators.
            security.consolidator = self.consolidate(
                security.symbol, Resolution.DAILY, self._consolidation_handler
            )
            
            # Warm up the indicators with historical data.
            warm_up_length = (
                max(self._std_period + 1, self._atr_period) 
                + self._training_set_duration.days
            )
            bars = self.history[TradeBar](
                security.symbol, warm_up_length, Resolution.DAILY
            )
            for bar in bars:
                security.consolidator.update(bar)

            self._contracts.append(security)

        for security in changes.removed_securities:
            # Remove the consolidator.
            self.subscription_manager.remove_consolidator(
                security.symbol, security.consolidator
            )
            # Reset the indicators.
            security.close_roc.reset()
            security.std_of_close_returns.reset()
            security.atr.reset()
            security.open_roc.reset()
            security.std_of_open_returns.reset()
            if security in self._contracts:
                self._contracts.remove(security)

    def _consolidation_handler(self, consolidated_bar):
        # Get the security object.
        security = self.securities[consolidated_bar.symbol]

        # Update the indicators and save their values.
        t = consolidated_bar.end_time
        if security.atr.update(consolidated_bar):
            security.indicator_history.loc[t, 'atr'] = \
                security.atr.current.value
        security.close_roc.update(t, consolidated_bar.close)
        if security.std_of_close_returns.is_ready:
            security.indicator_history.loc[t, 'std_of_close_returns'] = \
                security.std_of_close_returns.current.value
        security.open_roc.update(t, consolidated_bar.open)

        # Update the label history.
        if (security.std_of_open_returns.is_ready and 
            len(security.indicator_history.index) > self._future_std_period):
            security.label_history.loc[
                security.indicator_history.index[-self._future_std_period - 1]
            ] = security.std_of_open_returns.current.value
        
        # Trim the factor and label history.
        security.indicator_history = security.indicator_history[
            (security.indicator_history.index >= 
            self.time - self._training_set_duration)
        ]
        security.label_history = security.label_history[
            (security.label_history.index >= 
            self.time - self._training_set_duration)
        ]

In [11]:
import plotly.graph_objects as go
from datetime import datetime

# Equity curve data (cumulative return %)
curves = {
    "Ex11 — Ridge Regression": {
        "color": "#ef553b",
        "timestamps": [1546232400, 1546675197, 1547117994, 1547560791, 1548003588, 1548446385, 1548889182, 1549331979, 1549774776, 1550217573, 1550660370, 1551103167, 1551545964, 1551988761, 1552431558, 1552874355, 1553317152, 1553759950, 1554202747, 1554645544, 1555088341, 1555531138, 1555973935, 1556416732, 1556859529, 1557302326, 1557745123, 1558187920, 1558630717, 1559073514, 1559516311, 1559959108, 1560401905, 1560844702, 1561287500, 1561730297, 1562173094, 1562615891, 1563058688, 1563501485, 1563944282, 1564387079, 1564829876, 1565272673, 1565715470, 1566158267, 1566601064, 1567043861, 1567486658, 1567929455, 1568372252, 1568815050, 1569257847, 1569700644, 1570143441, 1570586238, 1571029035, 1571471832, 1571914629, 1572357426, 1572800223, 1573243020, 1573685817, 1574128614, 1574571411, 1575014208, 1575457005, 1575899802, 1576342600, 1576785397, 1577228194, 1577670991, 1578113788, 1578556585, 1578999382, 1579442179, 1579884976, 1580327773, 1580770570, 1581213367, 1581656164, 1582098961, 1582541758, 1582984555, 1583427352, 1583870150, 1584312947, 1584755744, 1585198541, 1585641338, 1586084135, 1586526932, 1586969729, 1587412526, 1587855323, 1588298120, 1588740917, 1589183714, 1589626511, 1590069308, 1590512105, 1590954902, 1591397700, 1591840497, 1592283294, 1592726091, 1593168888, 1593611685, 1594054482, 1594497279, 1594940076, 1595382873, 1595825670, 1596268467, 1596711264, 1597154061, 1597596858, 1598039655, 1598482452, 1598925250, 1599368047, 1599810844, 1600253641, 1600696438, 1601139235, 1601582032, 1602024829, 1602467626, 1602910423, 1603353220, 1603796017, 1604238814, 1604681611, 1605124408, 1605567205, 1606010003, 1606452800, 1606895597, 1607338394, 1607781191, 1608223988, 1608666785, 1609109582, 1609552379, 1609995176, 1610437973, 1610880770, 1611323567, 1611766364, 1612209161, 1612651958, 1613094755, 1613537553, 1613980350, 1614423147, 1614865944, 1615308741, 1615751538, 1616194335, 1616637132, 1617079929, 1617522726, 1617965523, 1618408320, 1618851117, 1619293914, 1619736711, 1620179508, 1620622305, 1621065103, 1621507900, 1621950697, 1622393494, 1622836291, 1623279088, 1623721885, 1624164682, 1624607479, 1625050276, 1625493073, 1625935870, 1626378667, 1626821464, 1627264261, 1627707058, 1628149855, 1628592653, 1629035450, 1629478247, 1629921044, 1630363841, 1630806638, 1631249435, 1631692232, 1632135029, 1632577826, 1633020623, 1633463420, 1633906217, 1634349014, 1634791811, 1635234608, 1635677405, 1636120203, 1636563000, 1637005797, 1637448594, 1637891391, 1638334188, 1638776985, 1639219782, 1639662579, 1640105376, 1640548173, 1640990970, 1641433767, 1641876564, 1642319361, 1642762158, 1643204955, 1643647753, 1644090550, 1644533347, 1644976144, 1645418941, 1645861738, 1646304535, 1646747332, 1647190129, 1647632926, 1648075723, 1648518520, 1648961317, 1649404114, 1649846911, 1650289708, 1650732505, 1651175303, 1651618100, 1652060897, 1652503694, 1652946491, 1653389288, 1653832085, 1654274882, 1654717679, 1655160476, 1655603273, 1656046070, 1656488867, 1656931664, 1657374461, 1657817258, 1658260056, 1658702853, 1659145650, 1659588447, 1660031244, 1660474041, 1660916838, 1661359635, 1661802432, 1662245229, 1662688026, 1663130823, 1663573620, 1664016417, 1664459214, 1664902011, 1665344808, 1665787606, 1666230403, 1666673200, 1667115997, 1667558794, 1668001591, 1668444388, 1668887185, 1669329982, 1669772779, 1670215576, 1670658373, 1671101170, 1671543967, 1671986764, 1672429561, 1672872358, 1673315156, 1673757953, 1674200750, 1674643547, 1675086344, 1675529141, 1675971938, 1676414735, 1676857532, 1677300329, 1677743126, 1678185923, 1678628720, 1679071517, 1679514314, 1679957111, 1680399908, 1680842706, 1681285503, 1681728300, 1682171097, 1682613894, 1683056691, 1683499488, 1683942285, 1684385082, 1684827879, 1685270676, 1685713473, 1686156270, 1686599067, 1687041864, 1687484661, 1687927458, 1688370256, 1688813053, 1689255850, 1689698647, 1690141444, 1690584241, 1691027038, 1691469835, 1691912632, 1692355429, 1692798226, 1693241023, 1693683820, 1694126617, 1694569414, 1695012211, 1695455008, 1695897806, 1696340603, 1696783400, 1697226197, 1697668994, 1698111791, 1698554588, 1698997385, 1699440182, 1699882979, 1700325776, 1700768573, 1701211370, 1701654167, 1702096964, 1702539761, 1702982558, 1703425356, 1703868153, 1704310950, 1704753747, 1705196544, 1705639341, 1706082138, 1706524935, 1706967732, 1707410529, 1707853326, 1708296123, 1708738920, 1709181717, 1709624514, 1710067311, 1710510109, 1710952906, 1711395703, 1711838500],
        "returns_pct": [0.0, 0.53, 2.03, 3.05, 6.59, 6.65, 7.89, 9.17, 8.73, 11.59, 14.28, 16.11, 15.56, 7.65, 8.6, 11.73, 11.78, 12.36, 18.05, 20.47, 21.02, 20.83, 21.93, 22.68, 21.8, 18.04, 12.04, 16.74, 13.97, 10.63, 3.21, 20.13, 22.13, 26.32, 29.37, 29.31, 32.32, 27.26, 35.23, 35.5, 34.9, 34.34, 22.87, 18.33, 17.67, 14.98, 9.87, 13.09, 15.56, 20.38, 23.74, 22.21, 23.21, 23.3, 14.61, 15.35, 22.97, 22.64, 23.24, 26.47, 28.91, 33.13, 34.43, 38.67, 36.12, 39.88, 34.41, 37.99, 41.05, 45.04, 46.24, 49.36, 48.84, 52.14, 54.17, 59.45, 54.98, 49.87, 49.62, 57.6, 65.6, 63.78, 43.68, 15.12, 13.35, -4.53, -14.27, -17.63, -17.68, -17.05, -18.28, -15.49, -15.1, -15.11, -14.63, -14.24, -13.98, -13.11, -14.09, -12.67, -11.01, -10.28, -4.44, -4.44, -7.1, -6.36, -6.27, -4.56, -3.18, -3.05, -2.28, -0.89, -2.73, -2.06, 0.11, 3.13, 2.66, 3.55, 6.08, 7.43, 5.21, 3.0, 4.79, 3.55, 3.47, 4.21, 4.27, 7.69, 7.45, 6.4, 3.78, 0.03, 7.28, 10.46, 13.4, 10.3, 13.62, 13.75, 15.07, 14.72, 15.98, 16.71, 16.73, 17.94, 19.52, 20.67, 18.61, 21.47, 16.64, 16.77, 19.24, 20.75, 21.49, 20.45, 16.37, 18.38, 20.65, 27.19, 31.15, 31.29, 31.94, 33.38, 37.42, 40.55, 42.72, 41.81, 42.01, 42.43, 47.89, 43.85, 38.51, 43.06, 45.53, 47.42, 45.78, 46.27, 39.71, 39.02, 39.75, 41.96, 43.28, 43.21, 40.42, 45.39, 45.58, 46.12, 47.98, 50.35, 47.93, 50.72, 52.64, 52.46, 50.45, 46.89, 46.97, 50.74, 42.39, 44.01, 48.93, 55.4, 57.82, 61.68, 62.76, 67.9, 65.53, 66.1, 64.21, 65.47, 54.48, 56.73, 65.47, 67.3, 65.08, 68.63, 70.96, 69.14, 65.27, 64.44, 50.32, 46.13, 48.93, 49.67, 50.02, 49.0, 43.5, 42.64, 42.71, 32.54, 34.61, 43.37, 43.92, 47.1, 44.89, 42.37, 35.56, 35.0, 27.11, 27.44, 21.02, 15.99, 13.32, 7.85, 7.41, 18.73, 17.19, 16.46, 0.92, -4.75, -4.53, -7.66, -8.81, -6.22, -9.68, -2.35, -2.37, 4.55, 4.81, 4.61, 11.85, 11.3, 5.62, -2.16, -8.16, -5.42, -9.59, -12.75, -13.96, -16.0, -10.17, -16.34, -15.19, -10.75, -2.84, 2.77, -1.62, 1.29, 6.39, 5.77, 8.61, 6.09, 9.8, 4.42, 3.78, -0.17, -0.3, -0.31, -0.56, 0.05, 1.8, 1.05, 1.63, 2.89, 4.42, 3.34, 5.08, 3.86, -3.05, -4.58, -2.2, -14.95, -14.98, -14.9, -14.99, -11.98, -11.51, -10.63, -10.35, -10.76, -10.89, -11.71, -12.14, -14.25, -13.13, -12.86, -12.71, -8.92, -9.74, -6.41, -2.05, -2.26, -1.88, 1.17, -2.64, 0.86, 5.93, 6.47, 8.94, 6.5, 3.84, 4.2, -5.05, -2.8, -1.5, 1.78, -0.2, 0.21, 2.19, 1.62, -2.88, -8.46, -3.67, -2.86, -2.9, -8.53, -13.74, -3.37, -1.6, 0.06, 3.69, 6.58, 8.11, 13.69, 14.04, 22.04, 23.21, 23.26, 24.47, 20.78, 22.81, 22.3, 21.7, 28.32, 29.68, 33.59, 35.27, 30.62, 34.37, 39.23, 37.79, 35.77, 36.12, 34.55, 35.33, 34.8, 37.0],
    },
}

fig = go.Figure()
for label, data in curves.items():
    dates = [datetime.fromtimestamp(t) for t in data['timestamps']]
    fig.add_trace(go.Scatter(
        x=dates, y=data['returns_pct'],
        mode='lines', name=label,
        line=dict(color=data['color'], width=2),
    ))

fig.update_layout(
    title="Ex11 — Ridge Regression",
    xaxis_title="Date",
    yaxis_title="Cumulative Return %",
    template="plotly_white",
    hovermode="x unified",
)
fig.show()

**References:**

- Hoerl, A. E. & Kennard, R. W. (1970). "Ridge Regression: Biased Estimation for Nonorthogonal Problems." *Technometrics*, 12(1), 55–67. — Seminal paper introducing ridge regularization; L2 penalty guarantees invertibility under multicollinearity.
- Hoerl, A. E. & Kennard, R. W. (1970). "Ridge Regression: Applications to Nonorthogonal Problems." *Technometrics*, 12(1), 69–82. — Companion paper with practical applications.
- Kim, J. H. (2019). "Multicollinearity and misleading statistical results." *Korean Journal of Anesthesiology*, 72(6), 558–569. [doi:10.4097/kja.19087](https://pmc.ncbi.nlm.nih.gov/articles/PMC6900425/). — Demonstrates that multicollinearity inflates coefficient variance, produces unreliable estimates, and can flip signs.
- Wilson, C.-A. (2025). "Explainable AI in Finance: Addressing the Needs of Diverse Stakeholders." CFA Institute Research & Policy Center. [doi:10.56227/25.1.25](https://rpc.cfainstitute.org/research/reports/2025/explainable-ai-in-finance). — Classifies linear regression as ante-hoc (built-in) explainability; identifies transparency as critical for regulatory compliance and risk management.
- Wilder, J. W. (1978). *New Concepts in Technical Trading Systems*. Trend Research. — Original ATR (Average True Range) definition.
- Murphy, J. J. (1999). *Technical Analysis of the Financial Markets*. New York Institute of Finance. Ch. 7. — Open interest interpretation: rising open interest signals new money flow and trend confirmation.
- Sharpe, W. F. (1966). "Mutual Fund Performance." *The Journal of Business*, 39(1), 119–138. — Sharpe ratio (reward-to-variability ratio) first defined here; used to evaluate all strategies.
- Jensen, M. C. (1968). "The Performance of Mutual Funds in the Period 1945–1964." *The Journal of Finance*, 23(2), 389–416. — Jensen's alpha: excess return after accounting for market exposure (CAPM).
- Schwab. [S&P 500 E-Mini Futures](https://www.schwab.com/futures/sp-500-emini). — E-Mini multiplier ($50 per index point); notional value context for position sizing.
- CME Group. [Corn Futures Contract Specs](https://www.cmegroup.com/markets/agriculture/grains/corn.contractSpecs.html). — 5,000 bushels per contract; notional value context.
- Pik, J. et al. (2025). *Hands-On AI Trading*. Wiley. pp. ~340–360. — Ex11 strategy design and implementation.

## Technique 2 — Temporal CNN (Deep Learning)

### Slide 7 — Technique #2: Deep Learning

Technique #2 — Adding Complexity

Temporal CNN: Predicting Price Direction

"Replace the linear model with a neural network. Does complexity help?"

---

Technique 2. We just saw what happens with a linear model — it couldn't find any signal the market wasn't already giving us. So now we're going to make a fundamental shift: from a model that can only learn straight-line relationships to one that can learn curves, interactions, and patterns that a human might never think to look for. We're replacing the ridge regression with a convolutional neural network — a CNN.

What is a CNN? Originally, convolutional neural networks were designed for image recognition — detecting edges, shapes, and objects in photos. But the core idea — sliding a small filter across an input to detect local patterns — works just as well on time series. Instead of scanning a 2D image for a cat's ear, we're scanning a 1D price sequence for things like 'three days of rising closes followed by a volume spike.' The key advantage over our linear model: a CNN can learn _nonlinear_ patterns. Ridge regression can only model 'if volatility goes up by X, the weight goes down by Y.' A CNN can learn 'if volatility is low _and_ volume is rising _and_ the trend was down last week, then do Z.' It captures interactions between features across time.

But not just any CNN — a _temporal_ CNN. That's the twist. It splits the extracted patterns into three time regions — long-term, mid-term, and short-term — and lets the model learn that different patterns matter at different time scales. Maybe what happened two weeks ago is irrelevant, but what happened yesterday is critical. The model decides.


### Slide 8 — Technique #2: The Architecture

Temporal CNN Architecture

[Figure 6.45 — Architecture Diagram]

```
Input: 15 days × 5 features (OHLCV)
↓
Conv1D(30 filters, kernel=4, ReLU)
↓
┌──────┼──────┐
Long   Mid    Short    ← 3-way temporal split
(old)  (mid)  (recent)
↓      ↓      ↓
1×1    1×1    1×1      ← Learn different weights per region
Conv   Conv   Conv
└──────┼──────┘
↓
Concatenate → Flatten → Dense(3, softmax)
↓
P(UP), P(DOWN), P(STATIONARY)
```

Key insight: The temporal split lets the model weight recent vs historical patterns differently.

---

Let me walk you through this architecture from top to bottom. The input is 15 trading days — three weeks — of OHLCV data. That's open, high, low, close, and volume. So the model sees a matrix: 15 rows by 5 columns. Each row is one day, each column is one feature. Think of it as a small spreadsheet that the model reads as a picture — it's looking for _patterns_ in how these five numbers change over time.

The first layer is a 1D convolution — Conv1D with 30 filters and a kernel size of 4. What does that mean? Imagine a small window — 4 days wide, covering all 5 features — that slides across the 15-day sequence one day at a time. At each position, it multiplies the data by a set of learned weights and produces a single number. That's one filter. We have 30 of these filters, each learning to detect a different 4-day pattern. One filter might learn to detect 'volume spiking while price is falling.' Another might detect 'three consecutive closes above the open.' After this layer, we have 12 positions — because a 4-day kernel on 15 days gives you 15 minus 4 plus 1 equals 12 — and 30 features per position. The ReLU activation zeros out any negative values, keeping only the patterns the model considers 'active.'

Now here's the clever part — the temporal split. Those 12 positions are divided into three equal regions of 4 positions each. The first four positions represent the _oldest_ data — what happened two to three weeks ago. The middle four are the _mid-term_ — roughly a week ago. The last four are the _most recent_ — the last few days. Each region passes through its own 1×1 convolution. A 1×1 convolution compresses the 30 feature channels down to 1, but it does so with its _own_ learned weights. This is the key insight: the model can learn that recent volume matters more than old volume, or that long-term price trends matter more than recent ones. Each time region gets its own perspective.

Finally, the three regions are concatenated back together — giving us 12 values — flattened into a single vector, and fed into a dense layer with 3 neurons and softmax activation. Softmax converts the raw outputs into probabilities that sum to 1.0. The three outputs are P(UP), P(DOWN), and P(STATIONARY) — the model's confidence that the stock is going up, down, or staying flat.


### Slide 9 — Technique #2: Trading Logic

How It Trades

- Universe: Top 3 QQQ ETF components by weight (weekly)
- Training: Every Monday — retrain CNN from scratch, 20 epochs, 500-day window
- Prediction: For each stock → P(UP), P(DOWN), P(STATIONARY)
- Confidence filter: Only trade when confidence > 55%
- Random chance = 33% (3 classes)
- 55% threshold = meaningful signal
- Sizing: Confidence-weighted directional allocation:

$wᵢ = dᵢ · cᵢ / Σⱼ |dⱼ · cⱼ|$

where dᵢ = direction (+1 or −1), cᵢ = confidence

- Execution: Full portfolio turnover each week

When uncertain → sit out. Capital preservation by design.

---

The trading logic is elegant. We take the top 3 QQQ ETF components by weight — and QQQ tracks the Nasdaq-100, so these are the largest mega-cap tech stocks. Think Apple, Microsoft, Nvidia — the exact names change week to week as ETF weights shift, but we're always trading the most heavily weighted constituents. Why only 3? Concentration. The model's signal is strongest for high-liquidity stocks with consistent volume, and a narrow universe means each prediction gets meaningful capital.

Every Monday morning, we retrain the CNN from scratch on each stock's trailing 500 days of price data. From scratch — no transfer learning, no warm-starting from last week's weights. Twenty epochs — that's 20 complete passes through the training data. An epoch means the model sees every training sample once. Why retrain weekly? Because markets aren't stationary — the patterns that predict price direction in March may not work in September. A 500-day window — about two years — gives the model enough history to learn stable patterns while still being responsive to recent market regimes.

Then we predict: is this stock going up, down, or staying flat over the next week? The model outputs three probabilities that sum to 1.0. The highest probability becomes the prediction, and that probability is the confidence score.

Here's the discipline: we only trade when the model's confidence exceeds 55%. Why 55%? With three classes, random guessing gives you a 1-in-3 chance — 33.3%. A model that's only 40% confident is barely better than a coin flip. The 55% threshold means the model must be roughly 22 percentage points above random chance before we risk any capital. This isn't arbitrary — it's a meaningful gap. If the model can't clear that bar, we sit out entirely. The algorithm goes to cash. This is capital preservation by design — during volatile or ambiguous markets, the strategy is simply uninvested.

When we do trade, the position size is proportional to confidence times direction. A 70% confidence UP prediction on Apple gets a weight of +0.70. A 56% confidence DOWN prediction on Microsoft gets a weight of −0.56. The weights are then normalized so the total absolute exposure doesn't exceed 1.0. The sign determines long versus short — this is the first strategy in our progression that can go short. The higher the confidence, the larger the position. And every Monday, the entire portfolio turns over — all existing positions are liquidated and rebuilt from the new predictions.


### Slide 10 — Technique #2: Results

Results: Temporal CNN on QQQ Stocks

Period: 2018-12-31 to 2024-04-01 · Starting capital: $100K

- Sharpe Ratio: 0.649 (3× the ridge model)
- CAGR: 18.6%
- Net Profit: +145.2%
- Alpha: 0.093 ✓ Genuine positive alpha
- Beta: 0.278 ✓ Low market exposure
- Max Drawdown: 26.3% (vs 54.7% for ridge)
- PSR: 21.9% (not statistically significant — but third-highest of all exercises)

The CNN generates real independent returns. Beta of 0.278 means it's not just riding the market.

---

And now the results change dramatically. Let's walk through these numbers and compare them to Technique 1. Sharpe ratio: 0.649 — three times the ridge model's 0.212. Remember, the Sharpe ratio measures return per unit of risk. We went from earning a fifth of a unit of return per unit of volatility to earning roughly two-thirds of a unit. Still below the 1.0 threshold for 'good,' but a massive improvement.

Net profit of 145% over five and a quarter years — a compound annual growth rate of 18.6%. Compare that to the ridge model's 5.85%. And this time we're not just tracking the market — over the same period, the S&P 500 returned roughly 130%. Our CNN returned 145% while taking _less_ systematic risk than the market. That's a fundamentally different proposition.

Now alpha and beta — and this is where the real story is. Alpha: 0.093. Positive. The model is generating returns that aren't explained by market movements. To put that in context, the ridge model had alpha of negative 0.062 — it was _destroying_ value after accounting for market exposure. The CNN is _creating_ value. And beta: 0.278. That's remarkably low — compare it to the ridge model's 1.146. A beta of 0.278 means that when the market moves 10%, this strategy only moves about 2.8% in sympathy. The rest of its returns come from the model's own signal. This is what quants call _market-neutral-ish_ — not perfectly hedged, but largely independent of whether the overall market goes up or down.

Why is the beta so low? It's the confidence threshold at work. The 55% filter means the strategy sits in cash during uncertain periods. When the market is volatile and direction is unclear, the model's confidence drops below 55% and the algorithm goes flat — uninvested. It effectively _self-hedges_ by refusing to play when the game is rigged against it.

Maximum drawdown: 26.3% — versus 54.7% for the ridge model. Half the pain. If you're managing a hundred thousand dollars, you'd see a twenty-six thousand dollar drawdown at worst. Painful, but recoverable. Compare that to losing more than half your capital with the ridge model.

Now the PSR — Probabilistic Sharpe Ratio — which I should explain. The PSR asks: given the observed Sharpe ratio and the number of data points, what's the probability that the _true_ Sharpe ratio is positive? It accounts for the possibility that a good Sharpe is just luck. Our PSR is 21.9% — meaning there's about a 1-in-5 chance the true Sharpe is actually positive. That's not statistically significant at any conventional threshold. It's the third-highest of all 19 exercises in the book, but it tells us we'd need a longer backtest to be confident these results aren't noise.

Look at the equity curve. Unlike the ridge model, which just tracked the broad market, this curve has its own character — it moves independently. The steady upward climb from 2020 through 2024 isn't just riding the post-COVID recovery. The low beta of 0.278 confirms it: the CNN has learned genuine patterns in price direction.

Takeaway: neural network complexity isn't wasted. The temporal split architecture captures real patterns that a linear model misses. We have genuine positive alpha, low market exposure, and half the drawdown risk. But it takes effort — weekly retraining, 20 epochs per stock, 500 days of data, roughly 780 training runs over a 5-year backtest. Can we do better with _less_ effort?


In [ ]:
# EX14 — main.py
# QuantConnect Algorithm Source
# This code runs on the QuantConnect platform
# (requires AlgorithmImports and QC runtime)

# region imports
from AlgorithmImports import *

import math

from temporalcnn import TemporalCNN, Direction, factor_names
# endregion


class TemporalCNNPredictionAlgorithm(QCAlgorithm):
 """
 This algorithm demonstration one way to apply Deep Learning 
 Classification in an attempt to forecast the movement of future 
 stock prices. Specifically, the strategy uses a Temporal 
 Convolutional Neural Network model to predict the direction of 
 future prices based on several samples of trailing OHLCV data.
 """

 def initialize(self):
  self.set_start_date(2018, 12, 31) 
  self.set_end_date(2024, 4, 1)
  self.set_cash(100_000)

  self._training_samples = self.get_parameter("training_samples", 500)
  self._universe_size = self.get_parameter("universe_size", 3)

  etf = Symbol.create("QQQ", SecurityType.EQUITY, Market.USA)
  date_rule = self.date_rules.week_start(etf)
  self.universe_settings.schedule.on(date_rule)
  self.universe_settings.data_normalization_mode = DataNormalizationMode.RAW
  self.universe_settings.asynchronous = True
  self._universe = self.add_universe(
   self.universe.etf(etf, universe_filter_func=self._select_assets)
  )

  self.train(date_rule, self.time_rules.at(9, 0), self._update_models)
  self.schedule.on(
   date_rule, self.time_rules.after_market_open(etf, 2), self._trade
  )

  # Load pre-trained and serialized models from the Object Store.
  if self.live_mode:
   self._models_by_symbol = {}
   self._key = 'cnn_models'
   if self.object_store.contains_key(self._key):
    self._models_by_symbol = json.loads(
     self.object_store.read(self._key)
    )

 def _select_assets(self, constituents):
  # Select the assets with the largest weight in the ETF.
  constituents = [c for c in constituents if c.weight]
  if constituents: 
   return [
    c.symbol 
    for c in sorted(
     constituents, key=lambda c: c.weight
    )[-self._universe_size:]
   ]
  return Universe.UNCHANGED

 def _trade(self):
  # Get predictions for all the assets.
  weight_by_symbol = {}
  for symbol in self._universe.selected:
   security = self.securities[symbol]
   symbol_df = security.history.tail(15)
   prediction, confidence = security.cnn.predict(symbol_df)
   if (prediction != Direction.STATIONARY and 
    not math.isnan(confidence) and 
    confidence > .55):
    factor = (-1 if prediction == Direction.DOWN else 1)
    weight_by_symbol[security.symbol] = factor * confidence
   self.plot("Confidence", str(security.symbol.id), confidence)
  
  # Calculate portfolio weights and rebalance.
  weight_sum = sum([abs(x) for x in weight_by_symbol.values()])
  weight_factor = 1 if weight_sum <= 1 else 1 / weight_sum
  portfolio_targets = [
   PortfolioTarget(symbol, weight * weight_factor) 
   for symbol, weight in weight_by_symbol.items()
  ]
  self.set_holdings(portfolio_targets, True)

 def on_splits(self, splits):
  for symbol, split in splits.items():
   if split.type == SplitType.SPLIT_OCCURRED:
    self._initialize_security(self.securities[symbol])

 def _initialize_security(self, security):
  symbol = security.symbol
  # Remove the old consolidator if there was one.
  if hasattr(security, 'consolidator'):
   self.subscription_manager.remove_consolidator(
    symbol, security.consolidator
   )
  # Add a new consolidator that adds bars to the asset history.
  security.consolidator = self.consolidate(
   symbol, Resolution.DAILY, self._consolidation_handler
  )
  # Warm up the history dataframe.
  security.history = self.history(
   symbol, self._training_samples, Resolution.DAILY, 
   data_normalization_mode=DataNormalizationMode.SCALED_RAW
  ).loc[symbol][factor_names]

 def _consolidation_handler(self, bar):
  security = self.securities[bar.symbol]
  security.history.loc[bar.end_time] = (
   bar.open, bar.high, bar.low, bar.close, bar.volume
  )
  security.history = security.history.iloc[-self._training_samples:]

 def on_securities_changed(self, changes):
  for security in changes.added_securities:
   serialized_model = None
   # If we're live trading and the algorithm was
   # re-deployed before `_update_models` runs, load
   # the pre-trained model from the last deployment.
   if self.live_mode:
    serialized_model = self._models_by_symbol.get(
     str(security.symbol.id), None
    )
   security.cnn = TemporalCNN(serialized_model)
   self._initialize_security(security)
  
  for security in changes.removed_securities:
   self.subscription_manager.remove_consolidator(
    security.symbol, security.consolidator
   )

 def _update_models(self):
  for symbol in self._universe.selected:
   security = self.securities[symbol] 
   model_json = security.cnn.train(security.history)
   if self.live_mode:
    self._models_by_symbol[str(symbol.id)] = model_json

 def on_end_of_algorithm(self):
  if self.live_mode:
   self.object_store.save(
    self._key, json.dumps(self._models_by_symbol)
   )

In [ ]:
# EX14 — temporalcnn.py
# QuantConnect Algorithm Source
# This code runs on the QuantConnect platform
# (requires AlgorithmImports and QC runtime)

#region imports
from AlgorithmImports import *

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Lambda, Flatten, Concatenate
from tensorflow.keras import Model
from tensorflow.keras import metrics
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import utils
from tensorflow.keras.models import model_from_json
from tensorflow.keras.config import enable_unsafe_deserialization
from tensorflow.keras.saving import register_keras_serializable
from sklearn.preprocessing import StandardScaler
import math
from keras.utils import set_random_seed
#endregion

set_random_seed(0)
enable_unsafe_deserialization()

# Define the variables used to make predictions.
factor_names = ['open', 'high', 'low', 'close', 'volume'] 


class Direction:
 """Constants used for labeling price movements."""
 # Labels must be integers because Keras (and most ML Libraries) 
 # only work with numbers.
 
 UP = 0
 DOWN = 1
 STATIONARY = 2


@register_keras_serializable()
def f0(x):
 return tf.split(x, num_or_size_splits=3, axis=1)[0]
 
@register_keras_serializable()
def f1(x):
 return tf.split(x, num_or_size_splits=3, axis=1)[1]

@register_keras_serializable()
def f2(x):
 return tf.split(x, num_or_size_splits=3, axis=1)[2]


class TemporalCNN:
 """Temporal Convolutional Neural Network Model built upon Keras."""
 
 # The name describes the architecture of the Neural Network model.
 # Temporal refers to the fact the layers are separated temporally 
 # into three regions. Convolutional refers to the fact Convolutional
 # layers are used to extract features.

 def __init__(self, model_json, n_tsteps=15):
  # n_tsteps is the number of time steps in time series for one 
  # input/prediction.
  self._n_tsteps = n_tsteps
  self._scaler = StandardScaler() # Used for Feature Scaling
  
  # Create the model.
  if model_json:
   # Load it from the Object Store.
   self._cnn = model_from_json(model_json)
  else:
   # Create a new one from scratch.
   self._cnn = self._create_model()

  # Compile the model.
  self._cnn.compile(
   optimizer='adam',
   loss=CategoricalCrossentropy(from_logits=True)
  )

 def _create_model(self):
  """Creates the neural network model."""
  inputs = Input(shape=(self._n_tsteps, len(factor_names)))
  
  # Extract features using a Convolutional layers ("CNN").
  feature_extraction = Conv1D(30, 4, activation='relu')(inputs)

  # Split layer into three regions based on time, ("Temporal").
  long_term = Lambda(f0, output_shape=(4, 30))(feature_extraction)
  mid_term = Lambda(f1, output_shape=(4, 30))(feature_extraction)
  short_term = Lambda(f2, output_shape=(4, 30))(feature_extraction)
  
  long_term_conv = Conv1D(1, 1, activation='relu')(long_term)
  mid_term_conv = Conv1D(1, 1, activation='relu')(mid_term)
  short_term_conv = Conv1D(1, 1, activation='relu')(short_term)
  
  # Combine the three layers back into one.
  combined = Concatenate(axis=1)(
   [long_term_conv, mid_term_conv, short_term_conv]
  )
  
  # Flattening is required since our input is a 2D matrix.
  flattened = Flatten()(combined)
  
  # 1 output neuron for each class (Up, Stationary, Down).
  # See the Direction class.
  outputs = Dense(3, activation='softmax')(flattened)
  
  # Specify the input and output layers of the model.
  return Model(inputs=inputs, outputs=outputs)
 
 def _prepare_data(
  self, data, rolling_avg_window_size=5, stationary_threshold=.0001):
  """Prepares the data for a format friendly for our model."""
  # rolling_avg_window_size is the window size for the future mid 
  # prices to average. This average is what the model wants to 
  # predict.
  # stationary_threshold is the maximum change of movement to be 
  # considered stationary for the average mid price stated above.
  df = data[factor_names]
  shift = -(rolling_avg_window_size - 1)
  
  # Define a function to label our data.
  def label_data(row):
   if row['close_avg_change_pct'] > stationary_threshold:
    return Direction.UP
   elif row['close_avg_change_pct'] < -stationary_threshold:
    return Direction.DOWN
   else:
    return Direction.STATIONARY
  
  # Compute the percentage change in the average of the close of 
  # the future 5 time steps at each time step.
  df['close_avg'] = df['close'].rolling(
   window=rolling_avg_window_size
  ).mean().shift(shift) 
  df['close_avg_change_pct'] = \
   (df['close_avg'] - df['close']) / df['close']
  
  # Label data based on direction.
  # axis=1 signifies a row-wise operation (axis=0 is col-wise).
  df['movement_labels'] = df.apply(label_data, axis=1)
  
  # Create lists to store each 2D input matrix and the 
  # corresponding label.
  data = []
  labels = []
  for i in range(len(df)-self._n_tsteps+1+shift):
   label = df['movement_labels'].iloc[i + self._n_tsteps - 1]
   data.append(df[factor_names].iloc[i:i + self._n_tsteps].values)
   labels.append(label)
  data = np.array(data)
  
  # Temporarily reshape the data to 2D since sklearn only works 
  # with 2D data.
  dim1, dim2, dim3 = data.shape
  data = data.reshape(dim1 * dim2, dim3)
  
  # Fit our scaler and transform our data in one method call.
  data = self._scaler.fit_transform(data)
  
  # Return the data to the original shape.
  data = data.reshape(dim1, dim2, dim3)
  
  # Keras needs dummy matrices for classification problems, hence 
  # the need for to_categorical(). num_classes ensures our dummy 
  # matrix has 3 columns, one for each label.
  return data, utils.to_categorical(labels, num_classes=3)

 def train(self, data):
  """Trains the model."""
  data, labels = self._prepare_data(data)
  self._cnn.fit(data, labels, epochs=20)
  return self._cnn.to_json()
 
 def predict(self, input_data):
  """
  Makes a prediction on the direction of the future stock 
  price.
  """
  input_data = self._scaler.transform(
   input_data.fillna(method='ffill').values
  )
  prediction = self._cnn.predict(input_data[np.newaxis, :])[0]
  direction = np.argmax(prediction)
  confidence = prediction[direction]
  return direction, confidence

In [20]:
import plotly.graph_objects as go
from datetime import datetime

# Equity curve data (cumulative return %)
curves = {
    "Ex14 — Temporal CNN": {
        "color": "#636efa",
        "timestamps": [1546232400, 1546675197, 1547117994, 1547560791, 1548003588, 1548446385, 1548889182, 1549331979, 1549774776, 1550217573, 1550660370, 1551103167, 1551545964, 1551988761, 1552431558, 1552874355, 1553317152, 1553759950, 1554202747, 1554645544, 1555088341, 1555531138, 1555973935, 1556416732, 1556859529, 1557302326, 1557745123, 1558187920, 1558630717, 1559073514, 1559516311, 1559959108, 1560401905, 1560844702, 1561287500, 1561730297, 1562173094, 1562615891, 1563058688, 1563501485, 1563944282, 1564387079, 1564829876, 1565272673, 1565715470, 1566158267, 1566601064, 1567043861, 1567486658, 1567929455, 1568372253, 1568815050, 1569257847, 1569700644, 1570143441, 1570586238, 1571029035, 1571471832, 1571914629, 1572357426, 1572800223, 1573243020, 1573685817, 1574128614, 1574571411, 1575014208, 1575457005, 1575899803, 1576342600, 1576785397, 1577228194, 1577670991, 1578113788, 1578556585, 1578999382, 1579442179, 1579884976, 1580327773, 1580770570, 1581213367, 1581656164, 1582098961, 1582541758, 1582984556, 1583427353, 1583870150, 1584312947, 1584755744, 1585198541, 1585641338, 1586084135, 1586526932, 1586969729, 1587412526, 1587855323, 1588298120, 1588740917, 1589183714, 1589626511, 1590069308, 1590512106, 1590954903, 1591397700, 1591840497, 1592283294, 1592726091, 1593168888, 1593611685, 1594054482, 1594497279, 1594940076, 1595382873, 1595825670, 1596268467, 1596711264, 1597154061, 1597596859, 1598039656, 1598482453, 1598925250, 1599368047, 1599810844, 1600253641, 1600696438, 1601139235, 1601582032, 1602024829, 1602467626, 1602910423, 1603353220, 1603796017, 1604238814, 1604681611, 1605124409, 1605567206, 1606010003, 1606452800, 1606895597, 1607338394, 1607781191, 1608223988, 1608666785, 1609109582, 1609552379, 1609995176, 1610437973, 1610880770, 1611323567, 1611766364, 1612209162, 1612651959, 1613094756, 1613537553, 1613980350, 1614423147, 1614865944, 1615308741, 1615751538, 1616194335, 1616637132, 1617079929, 1617522726, 1617965523, 1618408320, 1618851117, 1619293914, 1619736712, 1620179509, 1620622306, 1621065103, 1621507900, 1621950697, 1622393494, 1622836291, 1623279088, 1623721885, 1624164682, 1624607479, 1625050276, 1625493073, 1625935870, 1626378667, 1626821465, 1627264262, 1627707059, 1628149856, 1628592653, 1629035450, 1629478247, 1629921044, 1630363841, 1630806638, 1631249435, 1631692232, 1632135029, 1632577826, 1633020623, 1633463420, 1633906217, 1634349015, 1634791812, 1635234609, 1635677406, 1636120203, 1636563000, 1637005797, 1637448594, 1637891391, 1638334188, 1638776985, 1639219782, 1639662579, 1640105376, 1640548173, 1640990970, 1641433768, 1641876565, 1642319362, 1642762159, 1643204956, 1643647753, 1644090550, 1644533347, 1644976144, 1645418941, 1645861738, 1646304535, 1646747332, 1647190129, 1647632926, 1648075723, 1648518520, 1648961318, 1649404115, 1649846912, 1650289709, 1650732506, 1651175303, 1651618100, 1652060897, 1652503694, 1652946491, 1653389288, 1653832085, 1654274882, 1654717679, 1655160476, 1655603273, 1656046071, 1656488868, 1656931665, 1657374462, 1657817259, 1658260056, 1658702853, 1659145650, 1659588447, 1660031244, 1660474041, 1660916838, 1661359635, 1661802432, 1662245229, 1662688026, 1663130823, 1663573621, 1664016418, 1664459215, 1664902012, 1665344809, 1665787606, 1666230403, 1666673200, 1667115997, 1667558794, 1668001591, 1668444388, 1668887185, 1669329982, 1669772779, 1670215576, 1670658374, 1671101171, 1671543968, 1671986765, 1672429562, 1672872359, 1673315156, 1673757953, 1674200750, 1674643547, 1675086344, 1675529141, 1675971938, 1676414735, 1676857532, 1677300329, 1677743126, 1678185924, 1678628721, 1679071518, 1679514315, 1679957112, 1680399909, 1680842706, 1681285503, 1681728300, 1682171097, 1682613894, 1683056691, 1683499488, 1683942285, 1684385082, 1684827879, 1685270677, 1685713474, 1686156271, 1686599068, 1687041865, 1687484662, 1687927459, 1688370256, 1688813053, 1689255850, 1689698647, 1690141444, 1690584241, 1691027038, 1691469835, 1691912632, 1692355429, 1692798227, 1693241024, 1693683821, 1694126618, 1694569415, 1695012212, 1695455009, 1695897806, 1696340603, 1696783400, 1697226197, 1697668994, 1698111791, 1698554588, 1698997385, 1699440182, 1699882980, 1700325777, 1700768574, 1701211371, 1701654168, 1702096965, 1702539762, 1702982559, 1703425356, 1703868153, 1704310950, 1704753747, 1705196544, 1705639341, 1706082138, 1706524935, 1706967732, 1707410530, 1707853327, 1708296124, 1708738921, 1709181718, 1709624515, 1710067312, 1710510109, 1710952906, 1711395703, 1711838500],
        "returns_pct": [0.0, -1.33, 0.83, -0.1, -1.06, -0.28, 2.11, 3.04, 3.0, 3.32, 3.1, 1.96, 3.35, 3.47, 4.36, 6.07, 6.28, 5.82, 4.73, 6.3, 6.18, 6.15, 7.24, 10.14, 12.25, 12.36, 13.52, 14.94, 17.03, 17.03, 14.26, 14.08, 15.66, 15.9, 16.4, 17.74, 18.19, 18.99, 20.41, 21.78, 20.85, 19.38, 18.2, 13.85, 13.57, 13.07, 13.23, 13.38, 14.79, 16.69, 15.89, 14.99, 15.05, 13.39, 12.42, 12.87, 12.85, 13.42, 15.18, 13.45, 15.94, 16.52, 18.32, 19.31, 18.17, 20.17, 18.98, 21.11, 21.61, 21.53, 21.83, 19.97, 20.26, 22.37, 24.64, 26.02, 26.43, 26.95, 33.85, 31.93, 33.52, 30.3, 32.59, 41.52, 36.74, 57.01, 52.44, 32.77, 30.77, 30.35, 32.86, 37.66, 45.06, 44.7, 45.93, 48.89, 50.21, 50.55, 49.28, 50.03, 47.46, 48.09, 47.36, 56.45, 49.96, 51.39, 52.52, 57.45, 63.6, 68.45, 64.44, 67.32, 66.55, 76.11, 82.9, 82.69, 81.48, 86.26, 93.11, 94.77, 86.51, 78.11, 75.99, 76.01, 85.92, 92.96, 87.05, 95.77, 96.26, 94.6, 92.35, 83.46, 86.09, 79.36, 79.24, 75.64, 78.57, 75.73, 74.9, 73.23, 79.91, 82.46, 81.93, 83.64, 75.83, 77.82, 75.84, 76.81, 73.49, 80.33, 82.97, 81.12, 80.15, 73.91, 69.18, 67.1, 68.43, 70.02, 68.62, 69.94, 69.28, 69.41, 67.18, 66.04, 65.46, 65.66, 66.22, 61.62, 61.36, 61.27, 61.14, 63.15, 61.34, 61.74, 63.24, 66.33, 68.2, 68.1, 70.37, 74.31, 79.28, 78.4, 78.34, 82.34, 82.26, 82.25, 83.27, 85.9, 86.96, 88.29, 93.24, 93.77, 93.86, 91.09, 86.51, 85.06, 82.25, 82.13, 80.89, 78.55, 75.64, 75.55, 75.18, 77.89, 78.57, 77.42, 85.69, 86.57, 87.82, 88.39, 97.74, 100.4, 105.72, 109.3, 110.66, 101.83, 99.26, 98.62, 98.65, 105.1, 106.91, 107.16, 106.43, 109.02, 106.13, 108.76, 113.84, 109.38, 112.8, 129.04, 122.3, 120.24, 118.37, 117.71, 118.56, 120.47, 116.59, 118.36, 116.36, 118.48, 114.84, 106.39, 101.9, 98.98, 102.6, 100.22, 109.06, 111.08, 123.57, 129.66, 129.26, 138.38, 131.82, 130.7, 133.94, 127.95, 124.86, 126.4, 118.51, 116.54, 113.17, 112.91, 114.35, 116.46, 116.34, 118.79, 120.78, 128.79, 143.88, 137.58, 140.01, 138.65, 147.07, 151.79, 134.57, 136.98, 151.89, 153.38, 155.29, 145.44, 152.91, 153.7, 154.67, 155.95, 156.87, 154.6, 159.16, 152.74, 147.47, 145.25, 150.56, 149.47, 159.33, 159.45, 158.71, 159.6, 156.71, 163.1, 153.87, 155.24, 162.57, 159.28, 153.75, 161.81, 162.03, 164.21, 169.47, 168.78, 180.48, 183.52, 180.27, 180.4, 181.62, 184.17, 180.54, 172.17, 170.97, 174.28, 174.69, 172.45, 172.66, 168.42, 168.16, 167.26, 171.02, 161.14, 162.08, 162.08, 163.31, 164.41, 157.36, 155.74, 157.92, 158.29, 150.64, 156.14, 154.81, 154.74, 164.54, 159.26, 162.43, 159.29, 160.96, 161.04, 161.04, 149.22, 144.86, 144.54, 141.85, 140.37, 139.83, 139.79, 143.23, 136.52, 133.55, 133.27, 133.75, 128.86, 127.08, 122.78, 125.16, 127.74, 126.32, 122.22, 122.34, 121.4, 119.4, 131.99, 134.2, 132.67, 134.97, 134.95, 138.76, 150.02, 144.45],
    },
}

fig = go.Figure()
for label, data in curves.items():
    dates = [datetime.fromtimestamp(t) for t in data['timestamps']]
    fig.add_trace(go.Scatter(
        x=dates, y=data['returns_pct'],
        mode='lines', name=label,
        line=dict(color=data['color'], width=2),
    ))

fig.update_layout(
    title="Ex14 — Temporal CNN",
    xaxis_title="Date",
    yaxis_title="Cumulative Return %",
    template="plotly_white",
    hovermode="x unified",
)
fig.show()

**References:**

- LeCun, Y. et al. (1989). "Backpropagation Applied to Handwritten Zip Code Recognition." *Neural Computation*, 1(4), 541–551. — Foundational CNN paper; demonstrated backpropagation-trained CNNs for image classification.
- Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press. Ch. 6 & 9. — Neural networks learn nonlinear mappings (universal approximation theorem); convolution filter mechanics.
- Bailey, D. H. & López de Prado, M. (2012). "The Sharpe Ratio Efficient Frontier." *Journal of Risk*, 15(2), 3–44. — Probabilistic Sharpe Ratio (PSR): estimates P(true Sharpe > 0 | observed data), accounting for sample size and higher moments.
- Pik, J. et al. (2025). *Hands-On AI Trading*. Wiley. pp. ~400–420, Figure 6.45. — Ex14 temporal CNN architecture and implementation.

## Technique 3 — Amazon Chronos (Foundation Model)

### Slide 11 — Technique #3: Foundation Model

Technique #3 — The Foundation Model Era

Amazon Chronos: Zero Training, Real Results

"What if you could skip the training entirely?"

---

Technique 3. This is the main event. We just saw what happens when you add neural network complexity — the CNN found genuine alpha with low market exposure, but it took _work_. Weekly retraining, 20 epochs per stock, roughly 780 training runs over a five-year backtest. Now we're going to ask a radical question: what if you could skip the training entirely?

This is the foundation model era. A foundation model is a large model that's been pre-trained on massive amounts of data from many different domains, and then applied — without any additional training — to tasks it has never seen before. The idea comes from natural language processing: models like GPT and T5 learned the structure of language from billions of sentences, and then they could answer questions, translate, summarize — tasks they were never explicitly trained for. The same idea works for time series.

What if I told you there's a model that was _never trained on financial data_ — it learned temporal patterns from weather data, energy data, retail data — and it produces a higher Sharpe ratio than our carefully trained CNN? That's Amazon Chronos. Released by Amazon in March 2024, it's a transformer model that learned the _shape_ of time series patterns from billions of data points across domains. And we're going to use it off the shelf.


### Slide 12 — Technique #3: What Is Chronos?

Amazon Chronos — A Foundation Model for Time Series

What it is:
- A pre-trained transformer (T5 architecture, ~8M parameters)
- Trained on diverse time series: weather, energy, retail, economics
- Never trained on financial data

How it works:
1. Tokenization: Continuous values → 4,096 discrete tokens (uniform bins)
2. Processing: T5 encoder-decoder with self-attention + cross-attention
3. Output: Probability distribution over future values (not a point estimate)

The key insight: Temporal patterns are universal. A "spike then mean reversion" pattern looks the same whether it's temperature, sales, or stock prices.

---

Let me unpack this slide from the top. Chronos is built on T5 — Text-to-Text Transfer Transformer — the same architecture family as many large language models. T5 was created by Google in 2019 for natural language processing: it reads a sequence of tokens as input and generates a sequence of tokens as output. Every NLP task — translation, summarization, question answering — gets framed as 'text in, text out.' Chronos takes that same idea and applies it to numbers instead of words.

The model has about 8 million parameters — that's 8 million adjustable weights that were set during pre-training. For context, GPT-3 has 175 _billion_ parameters. Chronos-tiny is remarkably small, which is part of the story: you don't need a massive model to get useful forecasts.

Now, how does a text model process numbers? That's where tokenization comes in. You take a continuous value — say, a stock price of $152.37 — and you normalize it by dividing by the mean of the series. That gives you a scale-invariant number. Then you map it to one of 4,096 discrete bins — think of it as rounding to one of 4,096 possible values. Now the stock price is a 'token,' just like a word in a sentence. The model doesn't know it's looking at stock prices versus temperature readings — it just sees a sequence of tokens.

The architecture is an encoder-decoder. Let me explain what that means. The encoder reads the historical token sequence — your past price data — and builds an internal representation of the patterns it sees. It uses _self-attention_: each token looks at every other token in the history to decide which past data points are most relevant. 'Does what happened 10 days ago matter more than what happened yesterday?' The model learns to answer that question. The decoder then generates future tokens one at a time, using _cross-attention_ to look back at the encoder's representation. At each step, it doesn't output a single number — it outputs a probability distribution over all 4,096 bins. That distribution tells you not just 'I think the price will be around here,' but 'here's how confident I am across the full range of possibilities.'

Why does this work for finance if it was never trained on financial data? Because temporal patterns are universal. A sudden spike followed by mean reversion looks the same whether you're measuring temperature in Phoenix, electricity demand in Tokyo, or stock prices in New York. The self-attention mechanism learned these fundamental shapes — trends, cycles, spikes, mean reversion — from billions of data points across weather, energy, retail, and economics. When you show it stock prices, it recognizes patterns it's already seen in other domains.


### Slide 13 — Technique #3: Two Strategies

Two Strategies — Same Trading Logic

Strategy 1: Base Model (Zero Training)
- Load pre-trained `chronos-t5-tiny` from HuggingFace
- No fine-tuning. No training. Zero compute cost.
- Forecast 63 trading days (3 months) per asset

Strategy 2: Fine-Tuned (3 Steps)
- Take the base model
- Fine-tune on 1 year of equity price data
- Just 3 gradient steps (learning rate: 1e-5)
- Save to QC Object Store for persistence

Both strategies then:
- Select top 5 most liquid SPY constituents (by dollar volume)
- Optimize portfolio weights: maximize Sharpe ratio (SciPy SLSQP)
- No short selling, fully invested
- Rebalance quarterly

---

We test two variants, and the contrast between them is the heart of the foundation model story.

Strategy 1 is the purest test: take the model off the shelf — literally download it from HuggingFace, which is like an app store for AI models — give it stock prices it has never seen, and ask it to forecast 3 months ahead. Zero training. Zero compute cost beyond loading the model. Just load and predict. This is what 'zero-shot' means in AI: the model performs a task it was never explicitly trained for.

Strategy 2 is fine-tuned — but barely. Let me explain what fine-tuning means. The model arrives with 8 million pre-trained weights. Fine-tuning means we take those weights and nudge them slightly using our own data. How slightly? Three gradient steps. A gradient step is one update to the model's weights — the optimizer computes how wrong the model's predictions are on our data, calculates which direction to adjust each weight, and takes one small step in that direction. We do that three times. Three. For context, the CNN in Technique 2 trained for 20 _epochs_ — and each epoch means seeing the entire dataset once, which involves hundreds or thousands of gradient steps. We're doing three total.

The learning rate is one times ten to the negative five — 0.00001. The learning rate controls how big each step is. A large learning rate makes aggressive changes; a tiny one makes cautious adjustments. At 1e-5, we're barely moving the weights at all — just enough to tune the model's existing knowledge toward financial price patterns without overwriting what it learned during pre-training. We feed it one year of equity price data, it takes three cautious steps, and then we save the updated model to QC's Object Store — that's QuantConnect's cloud storage — so it persists between rebalance cycles.

Both strategies share the same trading logic downstream. We select the top 5 most liquid SPY constituents — and 'most liquid' means highest dollar volume, which is price times shares traded per day. Dollar volume is a better liquidity measure than raw volume because a million shares of a $5 stock is far less capital than a million shares of a $500 stock. In practice, this filter consistently selects mega-cap names like Apple, Microsoft, Nvidia, Amazon, and Meta — the stocks where you can deploy the most capital without moving the market.

The model forecasts 63 trading days — roughly 3 months — of future closing prices for each of these 5 stocks. Then we feed those forecasted equity curves into a SciPy optimizer that maximizes the forward-looking Sharpe ratio. This is Markowitz mean-variance optimization — real portfolio theory, published by Harry Markowitz in 1952, and still the foundation of modern portfolio construction. The idea: given expected returns and expected risk for each asset, find the combination of weights that gives you the best return per unit of risk. The constraints are practical: all weights must be non-negative — no short selling — and they must sum to 1 — you're fully invested. The entire portfolio rebalances quarterly, not weekly like the CNN. Why quarterly? Because the model forecasts 3 months ahead, so there's no new information to act on until the next forecast window.


### Slide 14 — Technique #3: Portfolio Optimization

Portfolio Optimization — The Quant Math

Objective: Maximize the Sharpe ratio of the forecasted portfolio

$max_w  (R_p − R_f) / σ_p$

Subject to:
- Full investment: Σ wᵢ = 1
- No short selling: 0 ≤ wᵢ ≤ 1

Where:

$R_p = Σ wᵢ · r̄ᵢ · 252$

(annualized return from Chronos forecasts)

$σ_p = √(w^T Σ w · 252)$

(annualized risk from forecast covariance)

R_f = FOMC primary credit rate

Solver: SciPy SLSQP (Sequential Least Squares Programming)

---

This slide is for the quants in the room — but I'll walk everyone through it. The portfolio optimization is a constrained maximization problem. We want to find the set of weights — how much to allocate to each of the 5 stocks — that maximizes the forward-looking Sharpe ratio using the Chronos forecasts as our crystal ball.

Let me walk through the equations. The objective function at the top says: maximize, over all possible weight combinations w, the quantity R_p minus R_f, divided by sigma_p. R_p is the portfolio's expected return. R_f is the risk-free rate — what you'd earn by just holding government bonds. Sigma_p is the portfolio's expected volatility — how much the portfolio value is expected to bounce around. The Sharpe ratio is this entire fraction: excess return divided by risk. We want the combination of stocks that gives us the most return per unit of risk.

Now the components. R_p — portfolio return — equals the sum of each weight times each stock's expected daily return, times 252. Why 252? That's the number of trading days in a year. We multiply by 252 to _annualize_ the daily returns — converting a daily figure like 0.04% into a yearly figure like roughly 10%. Without annualization, you'd be comparing tiny daily numbers, which makes the optimization numerically unstable.

Sigma_p — portfolio risk — is the square root of w-transpose times the covariance matrix times w, times 252. The covariance matrix is the key object here. It's a grid of numbers that captures not just how volatile each stock is on its own, but how they move _together_. If Apple and Microsoft tend to go up on the same days and down on the same days — high covariance — then holding both doesn't diversify your risk much. If two stocks move independently — low covariance — holding both reduces your overall volatility. The optimizer uses this information to find weight combinations where the stocks' movements partially cancel each other out.

R_f is the FOMC primary credit rate — that's the Federal Reserve's benchmark interest rate at the time of each rebalance. It represents the opportunity cost of capital: if you can earn 5% risk-free, your portfolio needs to beat 5% before the Sharpe ratio even turns positive.

The constraints are practical. Full investment: all weights must sum to 1 — every dollar is allocated. No short selling: every weight is between 0 and 1 — you can only buy, not bet against. These are sensible constraints for a long-only portfolio.

The solver — SciPy SLSQP — is Sequential Least Squares Programming. It's a numerical optimization algorithm that handles both the equality constraint (weights sum to 1) and the bound constraints (no negative weights) simultaneously. You give it a starting guess — equal weights across all 5 stocks — and it iteratively adjusts until it finds the combination that maximizes the Sharpe ratio. The implementation actually _minimizes_ the _negative_ Sharpe ratio, because SciPy's minimize function only knows how to go downhill.

What makes this interesting is the combination: cutting-edge AI forecasting meets classical portfolio theory from 1952. Chronos provides the forward-looking return estimates, and Markowitz optimization decides how to allocate. The AI replaces human judgment about _where the market is going_; the math handles _how much to bet_.


### Slide 15 — Technique #3: Results Side by Side

Results: Chronos Base vs Fine-Tuned

Period: 2019-01-01 to 2024-04-01 · Starting capital: $100K

|                  | Base (0 training) | Fine-Tuned (3 steps) |
|------------------|--------------------|----------------------|
| Sharpe       | 0.727              | 0.846                |
| CAGR         | 23.2%              | 28.0%                |
| Net Profit   | +200%              | +266%                |
| Alpha        | 0.040              | 0.076                |
| Beta         | 1.125              | 1.110                |
| Max Drawdown | 41.2%              | 49.5%                |
| Win Rate     | 72%                | 72%                  |
| Capacity     | $1.4B              | $920M                |

Zero training = Sharpe 0.727. Three fine-tuning steps add +0.12 Sharpe but increase drawdown by 8 percentage points.

---

Let this sink in. Let's walk through these numbers the same way we did for the previous techniques, starting with the base model — zero training on financial data.

Sharpe ratio: 0.727. Compare that to the ridge model's 0.212 and the CNN's 0.649. With _no training at all_, the foundation model produces a higher risk-adjusted return than a neural network we retrained every week for five years. Net profit: 200% — your hundred thousand becomes three hundred thousand. CAGR: 23.2%. For reference, the CNN delivered 18.6% and the ridge model 5.85%. The progression is clear.

Now fine-tuning. Three gradient steps push the Sharpe to 0.846 — the highest of any strategy in our progression. Net profit: 266%. CAGR: 28%. Alpha nearly doubles, from 0.04 to 0.076. Remember what alpha means: it's the return that can't be explained by market movements. The fine-tuned model is generating almost twice the independent value of the base model, just from three cautious weight adjustments.

But — and this is an important 'but' — look at the drawdown column. The base model's maximum drawdown is 41.2%. The fine-tuned model's is 49.5% — at one point, nearly half the portfolio value evaporated before recovering. That's an 8 percentage point increase in downside risk for the fine-tuned variant. Why? Fine-tuning makes the model more aggressive. It becomes more confident in its predictions, which means larger position sizes, which means bigger swings in both directions. The fine-tuned model captures more upside _and_ more downside. There's no free lunch — the additional 66 percentage points of net profit come with additional pain.

Now beta. Both models have beta above 1.1 — the base at 1.125, fine-tuned at 1.110. This is fundamentally different from the CNN's 0.278. These Chronos strategies are _long the market_. Why? The universe is the top 5 most liquid stocks — mega-cap growth names like Apple, Microsoft, Nvidia. The strategy can't short, must be fully invested, and holds concentrated positions in the very stocks that _are_ the market. A beta above 1.0 is almost inevitable with this design. So much of the raw return is coming from market exposure, not from the model's forecasting skill.

This is the key nuance: alpha and returns are not the same thing. The CNN had _higher_ alpha — 0.093 versus Chronos fine-tuned's 0.076 — with _much lower_ beta. If you're building a market-neutral hedge fund, the CNN's signal is actually more valuable. But if you're building a long-biased portfolio and you care about total returns with minimal effort, Chronos wins decisively.

The win rate is identical: 72% for both variants. That means roughly three out of every four quarterly rebalances end up profitable. Consistent, but remember — with quarterly rebalancing, we only have about 20 data points over five years. That's a small sample.

Capacity is remarkable: $1.4 billion for the base model, $920 million for fine-tuned. That's how much capital you could deploy before your trades start moving the market. These are institutionally relevant numbers, and they come from trading only the most liquid stocks on Earth. The fine-tuned model has slightly lower capacity — it concentrates into fewer distinct positions, so each trade is larger relative to the stock's volume.

Look at the equity curves. Unlike the CNN's independently moving curve, these track the broad market — you can see the COVID crash in March 2020, the 2021–2022 tech run-up, the 2022 drawdown. That's the beta at work. But within that market shape, the fine-tuned curve consistently sits above the base curve — that's the alpha from fine-tuning. Three gradient steps, visible in the equity curve.

Takeaway: a foundation model with zero training beats a purpose-built CNN on Sharpe ratio and total returns. Three steps of fine-tuning make it even better, but at the cost of higher drawdown. The question for the next slide is: how does this all fit together?


In [ ]:
# EX18 BASE — main.py
# QuantConnect Algorithm Source
# This code runs on the QuantConnect platform
# (requires AlgorithmImports and QC runtime)

# region imports
from AlgorithmImports import *

import torch
from chronos import ChronosPipeline
from scipy.optimize import minimize
from transformers import set_seed 
# endregion

class HuggingFaceBaseModelDemo(QCAlgorithm):
    """
    This algorithm demonstrates how to use a pre-trained HuggingFace 
    model. It uses the "amazon/chronos-t5-tiny" model to forecast the 
    future equity curves of the 5 most liquid assets in the market,
    then it uses the SciPy package to find the portfolio weights
    that will maximize the future Sharpe ratio of the portfolio. 
    The portfolio is rebalanced every 3 months.
    """

    def initialize(self):
        self.set_start_date(2019, 1, 1)
        self.set_end_date(2024, 4, 1)
        self.set_cash(100_000)

        # Disable the daily precise end time because of the time rules.
        self.settings.daily_precise_end_time = False
        self.settings.min_absolute_portfolio_target_percentage = 0

        # Enable reproducibility.
        set_seed(1, True)

        # Load the pre-trained model.
        self._pipeline = ChronosPipeline.from_pretrained(
            "amazon/chronos-t5-tiny",
            device_map="cuda" if torch.cuda.is_available() else "cpu",
            torch_dtype=torch.bfloat16,
        )

        # Define the universe.
        spy = Symbol.create("SPY", SecurityType.EQUITY, Market.USA)
        self.universe_settings.schedule.on(self.date_rules.month_start(spy))
        self.universe_settings.resolution = Resolution.DAILY
        self._universe = self.add_universe(
            self.universe.dollar_volume.top(
                self.get_parameter('universe_size', 5)
            )
        )

        # Define some trading parameters.
        self._lookback_period = timedelta(
            365 * self.get_parameter('lookback_years', 1)
        )
        self._prediction_length = 3*21 # Three months of trading days

        # Schedule rebalances.
        self._last_rebalance = datetime.min
        self.schedule.on(
            self.date_rules.month_start(spy, 1), 
            self.time_rules.midnight, 
            self._trade
        )

        # Add warm up so the algorithm trades on deployment.
        self.set_warmup(timedelta(31))

    def on_warmup_finished(self):
        # Trade right after warm up is done.
        self._trade()

    def _sharpe_ratio(
            self, weights, returns, risk_free_rate, trading_days_per_year=252):
        # Define how to calculate the Sharpe ratio so we can use
        # it to optimize the portfolio weights.

        # Calculate the annualized returns and covariance matrix.
        mean_returns = returns.mean() * trading_days_per_year 
        cov_matrix = returns.cov() * trading_days_per_year

        # Calculate the Sharpe ratio.
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
        
        # Return negative Sharpe ratio because we minimize this
        # function in optimization.
        return -sharpe_ratio

    def _optimize_portfolio(self, equity_curves):
        returns = equity_curves.pct_change().dropna()
        num_assets = returns.shape[1]
        initial_guess = num_assets * [1. / num_assets,]
        # Find portfolio weights that maximize the forward Sharpe
        # ratio.
        result = minimize(
            self._sharpe_ratio, 
            initial_guess, 
            args=(
                returns,
                self.risk_free_interest_rate_model.get_interest_rate(self.time)
            ), 
            method='SLSQP', 
            bounds=tuple((0, 1) for _ in range(num_assets)), 
            constraints=(
                {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
            )
        ) 
        return result.x

    def _trade(self):
        # Don't rebalance during warm-up.
        if self.is_warming_up:
            return
        # Only rebalance on a quarterly basis.
        if self.time - self._last_rebalance < timedelta(80):
            return 
        self._last_rebalance = self.time

        symbols = list(self._universe.selected)

        # Get historical equity curves.
        history = self.history(symbols, self._lookback_period)['close'].unstack(0)
        
        # Forecast the future equity curves.
        all_forecasts = self._pipeline.predict(
            [
                torch.tensor(history[symbol].dropna()) 
                for symbol in symbols
            ], 
            self._prediction_length
        )
        
        # Take the median forecast for each asset.
        forecasts_df = pd.DataFrame(
            {
                symbol: np.quantile(
                    all_forecasts[i].numpy(), 0.5, axis=0 # 0.5 = median
                )
                for i, symbol in enumerate(symbols)
            }
        )

        # Find the weights that maximize the forward Sharpe 
        # ratio of the portfolio.
        optimal_weights = self._optimize_portfolio(forecasts_df)

        # Rebalance the portfolio.
        self.set_holdings(
            [
                PortfolioTarget(symbol, optimal_weights[i])
                for i, symbol in enumerate(symbols)
            ], 
            True
        )

In [ ]:
# EX18 FT — main.py
# QuantConnect Algorithm Source
# This code runs on the QuantConnect platform
# (requires AlgorithmImports and QC runtime)

# region imports
from AlgorithmImports import *

from scipy.optimize import minimize

import sys
from unittest.mock import MagicMock
# Patch missing typer_config dependency before importing chronos training internals
if 'typer_config' not in sys.modules:
    sys.modules['typer_config'] = MagicMock()

import torch
from ast import literal_eval
from pathlib import Path
from functools import partial
from transformers import Trainer, TrainingArguments, set_seed 
from gluonts.dataset.pandas import PandasDataset
from gluonts.itertools import Filter
from chronos import ChronosConfig, ChronosPipeline
from chronos.scripts.training.train import ChronosDataset, has_enough_observations, load_model
from chronos.scripts.training import train
from logging import getLogger, INFO
# endregion

class HuggingFaceFineTunedDemo(QCAlgorithm):
    """
    This algorithm demonstrates how to fine-tune a HuggingFace model.
    It uses the "amazon/chronos-t5-tiny" model to forecast the 
    future equity curves of the 5 most liquid assets in the market,
    then it uses the SciPy package to find the portfolio weights
    that will maximize the future Sharpe ratio of the portfolio. 
    The model is retrained and the portfolio is rebalanced every 3 
    months.
    """

    def initialize(self):
        self.set_start_date(2019, 1, 1)
        self.set_end_date(2024, 4, 1)
        self.set_cash(100_000)

        # Disable the daily precise end time because of the time rules.
        self.settings.daily_precise_end_time = False
        self.settings.min_absolute_portfolio_target_percentage = 0

        # Define the universe.
        spy = Symbol.create("SPY", SecurityType.EQUITY, Market.USA)
        self.universe_settings.schedule.on(self.date_rules.month_start(spy))
        self.universe_settings.resolution = Resolution.DAILY
        self._universe = self.add_universe(
            self.universe.dollar_volume.top(
                self.get_parameter('universe_size', 5)
            )
        )

        # Define some trading parameters.
        self._lookback_period = timedelta(
            365 * self.get_parameter('lookback_years', 1)
        )
        self._prediction_length = 3*21 # Three months of trading days

        # Schedule rebalances.
        self._last_rebalance = datetime.min
        self.schedule.on(
            self.date_rules.month_start(spy, 1), 
            self.time_rules.midnight, 
            self._trade
        )
        
        # Add warm up so the algorithm trades on deployment.
        self.set_warm_up(timedelta(31))

        # Define the model and some of its settings.
        self._device_map = "cuda" if torch.cuda.is_available() else "cpu"
        self._optimizer = 'adamw_torch_fused' if torch.cuda.is_available() else 'adamw_torch'
        self._model_name = "amazon/chronos-t5-tiny"
        self._model_path = self.object_store.get_file_path(
            f"llm/fine-tune/{self._model_name.replace('/', '-')}/"
        )

    def on_warmup_finished(self):
        # Trade right after warm up is done.
        self._trade()

    def _sharpe_ratio(
            self, weights, returns, risk_free_rate, trading_days_per_year=252):
        # Define how to calculate the Sharpe ratio so we can use
        # it to optimize the portfolio weights.

        # Calculate the annualized returns and covariance matrix.
        mean_returns = returns.mean() * trading_days_per_year 
        cov_matrix = returns.cov() * trading_days_per_year

        # Calculate the Sharpe ratio.
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
        
        # Return negative Sharpe ratio because we minimize this
        # function in optimization.
        return -sharpe_ratio

    def _optimize_portfolio(self, equity_curves):
        returns = equity_curves.pct_change().dropna()
        num_assets = returns.shape[1]
        initial_guess = num_assets * [1. / num_assets,]
        # Find portfolio weights that maximize the forward Sharpe
        # ratio.
        result = minimize(
            self._sharpe_ratio, 
            initial_guess, 
            args=(
                returns,
                self.risk_free_interest_rate_model.get_interest_rate(self.time)
            ), 
            method='SLSQP', 
            bounds=tuple((0, 1) for _ in range(num_assets)), 
            constraints=(
                {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
            )
        ) 
        return result.x

    def _trade(self):
        # Don't rebalance during warm-up.
        if self.is_warming_up:
            return
        # Only rebalance on a quarterly basis.
        if self.time - self._last_rebalance < timedelta(80):
            return 
        self._last_rebalance = self.time

        symbols = list(self._universe.selected)

        # Get historical equity curves.
        history = self.history(symbols, self._lookback_period)['close'].unstack(0)

        # Gather the training data.
        training_data_by_symbol = {}
        for symbol in symbols:
            df = history[[symbol]].dropna()
            if df.shape[0] < 10: # Skip this asset if there is very little data
                continue
            adjusted_df = df.reset_index()[['time', symbol]]
            # Use positional rename — str(symbol.id) may not match column name in current QC
            adjusted_df.columns = ['time', 'target']
            adjusted_df['time'] = pd.to_datetime(adjusted_df['time'])
            adjusted_df.set_index('time', inplace=True)
            adjusted_df = adjusted_df.resample('D').asfreq()
            training_data_by_symbol[symbol] = adjusted_df
        tradable_symbols = list(training_data_by_symbol.keys())
        
        # Fine-tune the model.
        output_dir_path = self._train_chronos(
            list(training_data_by_symbol.values()),
            context_length=int(252/2), # 6 months
            prediction_length=self._prediction_length,
            optim=self._optimizer,
            model_id=self._model_name,
            output_dir=self._model_path,
            learning_rate=1e-5,
            # Requires Ampere GPUs (e.g., A100)
            tf32=False,
            max_steps=3,
            # QC container fix: multiprocess DataLoader workers crash
            dataloader_num_workers=0,
            torch_compile=False
        )

        # Load the fine-tuned model.
        pipeline = ChronosPipeline.from_pretrained(
            output_dir_path,
            device_map=self._device_map,
            torch_dtype=torch.bfloat16,
        )

        # Forecast the future equity curves.
        all_forecasts = pipeline.predict(
            [
                torch.tensor(history[symbol].dropna())
                for symbol in tradable_symbols
            ], 
            self._prediction_length
        )

        # Take the median forecast for each asset.
        forecasts_df = pd.DataFrame(
            {
                symbol: np.quantile(
                    all_forecasts[i].numpy(), 0.5, axis=0 # 0.5 = median
                )
                for i, symbol in enumerate(tradable_symbols)
            }
        )

        # Find the weights that maximize the forward Sharpe 
        # ratio of the portfolio.
        optimal_weights = self._optimize_portfolio(forecasts_df)

        # Rebalance the portfolio.
        self.set_holdings(
            [
                PortfolioTarget(symbol, optimal_weights[i])
                for i, symbol in enumerate(tradable_symbols)
            ], 
            True
        )

    def _train_chronos(
            self, training_data,
            probability: Optional[str] = None,
            context_length: int = 512,
            prediction_length: int = 64,
            min_past: int = 64,
            max_steps: int = 200_000,
            save_steps: int = 50_000,
            log_steps: int = 500,
            per_device_train_batch_size: int = 32,
            learning_rate: float = 1e-3,
            optim: str = "adamw_torch_fused",
            shuffle_buffer_length: int = 100,
            gradient_accumulation_steps: int = 2,
            model_id: str = "google/t5-efficient-tiny",
            model_type: str = "seq2seq",
            random_init: bool = False,
            tie_embeddings: bool = False,
            output_dir: str = "./output/",
            tf32: bool = True,
            torch_compile: bool = True,
            tokenizer_class: str = "MeanScaleUniformBins",
            tokenizer_kwargs: str = "{'low_limit': -15.0, 'high_limit': 15.0}",
            n_tokens: int = 4096,
            n_special_tokens: int = 2,
            pad_token_id: int = 0,
            eos_token_id: int = 1,
            use_eos_token: bool = True,
            lr_scheduler_type: str = "linear",
            warmup_ratio: float = 0.0,
            dataloader_num_workers: int = 1,
            max_missing_prop: float = 0.9,
            num_samples: int = 20,
            temperature: float = 1.0,
            top_k: int = 50,
            top_p: float = 1.0):

        # Set up logging for the train object.
        train.logger = getLogger()
        train.logger.setLevel(INFO)
        # Ensure output_dir is a Path object.
        output_dir = Path(output_dir)
        # Convert probability from string to a list, or set default if 
        # None.
        if isinstance(probability, str):
            probability = literal_eval(probability)
        elif probability is None:
            probability = [1.0 / len(training_data)] * len(training_data)
        # Convert tokenizer_kwargs from string to a dictionary.
        if isinstance(tokenizer_kwargs, str):
            tokenizer_kwargs = literal_eval(tokenizer_kwargs)
        # Enable reproducibility.
        set_seed(1, True)
        # Create datasets for training, filtered by criteria.
        train_datasets = [
            Filter(
                partial(
                    has_enough_observations,
                    min_length=min_past + prediction_length,
                    max_missing_prop=max_missing_prop,
                ),
                PandasDataset(data_frame, freq="D"),
            )
            for data_frame in training_data
        ]
        # Load the model with the specified configuration.
        model = load_model(
            model_id=model_id,
            model_type=model_type,
            vocab_size=n_tokens,
            random_init=random_init,
            tie_embeddings=tie_embeddings,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
        )
        # Define the configuration for the Chronos 
        # tokenizer and other settings.
        chronos_config = ChronosConfig(
            tokenizer_class=tokenizer_class,
            tokenizer_kwargs=tokenizer_kwargs,
            n_tokens=n_tokens,
            n_special_tokens=n_special_tokens,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
            use_eos_token=use_eos_token,
            model_type=model_type,
            context_length=context_length,
            prediction_length=prediction_length,
            num_samples=num_samples,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        )

        # Add extra items to model config so that 
        # it's saved in the ckpt.
        model.config.chronos_config = chronos_config.__dict__
        # Create a shuffled training dataset with the 
        # specified parameters.
        shuffled_train_dataset = ChronosDataset(
            datasets=train_datasets,
            probabilities=probability,
            tokenizer=chronos_config.create_tokenizer(),
            context_length=context_length,
            prediction_length=prediction_length,
            min_past=min_past,
            mode="training",
        ).shuffle(shuffle_buffer_length=shuffle_buffer_length)

        # Define the training arguments.
        training_args = TrainingArguments(
            output_dir=str(output_dir),
            per_device_train_batch_size=per_device_train_batch_size,
            learning_rate=learning_rate,
            lr_scheduler_type=lr_scheduler_type,
            warmup_ratio=warmup_ratio,
            optim=optim,
            logging_dir=str(output_dir / "train-logs"),
            logging_strategy="steps",
            logging_steps=log_steps,
            save_strategy="steps",
            save_steps=save_steps,
            report_to=["tensorboard"],
            max_steps=max_steps,
            gradient_accumulation_steps=gradient_accumulation_steps,
            dataloader_num_workers=dataloader_num_workers,
            tf32=tf32, # remove this if not using Ampere GPUs (e.g., A100)
            torch_compile=torch_compile,
            ddp_find_unused_parameters=False,
            remove_unused_columns=False,
        )

        # Create a Trainer instance for training the model.
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=shuffled_train_dataset,
        )
        # Start the training process.
        trainer.train()
        # Save the trained model to the output directory.
        model.save_pretrained(output_dir)
        # Return the path to the output directory.
        return output_dir

In [30]:
import plotly.graph_objects as go
from datetime import datetime

# Equity curve data (cumulative return %)
curves = {
    "Ex18 — Chronos Base": {
        "color": "#00cc96",
        "timestamps": [1546318800, 1546761597, 1547204394, 1547647191, 1548089988, 1548532785, 1548975582, 1549418379, 1549861176, 1550303973, 1550746770, 1551189567, 1551632364, 1552075161, 1552517958, 1552960755, 1553403552, 1553846350, 1554289147, 1554731944, 1555174741, 1555617538, 1556060335, 1556503132, 1556945929, 1557388726, 1557831523, 1558274320, 1558717117, 1559159914, 1559602711, 1560045508, 1560488305, 1560931103, 1561373900, 1561816697, 1562259494, 1562702291, 1563145088, 1563587885, 1564030682, 1564473479, 1564916276, 1565359073, 1565801870, 1566244667, 1566687464, 1567130261, 1567573058, 1568015856, 1568458653, 1568901450, 1569344247, 1569787044, 1570229841, 1570672638, 1571115435, 1571558232, 1572001029, 1572443826, 1572886623, 1573329420, 1573772217, 1574215014, 1574657811, 1575100609, 1575543406, 1575986203, 1576429000, 1576871797, 1577314594, 1577757391, 1578200188, 1578642985, 1579085782, 1579528579, 1579971376, 1580414173, 1580856970, 1581299767, 1581742564, 1582185362, 1582628159, 1583070956, 1583513753, 1583956550, 1584399347, 1584842144, 1585284941, 1585727738, 1586170535, 1586613332, 1587056129, 1587498926, 1587941723, 1588384520, 1588827317, 1589270115, 1589712912, 1590155709, 1590598506, 1591041303, 1591484100, 1591926897, 1592369694, 1592812491, 1593255288, 1593698085, 1594140882, 1594583679, 1595026476, 1595469273, 1595912070, 1596354867, 1596797665, 1597240462, 1597683259, 1598126056, 1598568853, 1599011650, 1599454447, 1599897244, 1600340041, 1600782838, 1601225635, 1601668432, 1602111229, 1602554026, 1602996823, 1603439620, 1603882418, 1604325215, 1604768012, 1605210809, 1605653606, 1606096403, 1606539200, 1606981997, 1607424794, 1607867591, 1608310388, 1608753185, 1609195982, 1609638779, 1610081576, 1610524373, 1610967171, 1611409968, 1611852765, 1612295562, 1612738359, 1613181156, 1613623953, 1614066750, 1614509547, 1614952344, 1615395141, 1615837938, 1616280735, 1616723532, 1617166329, 1617609126, 1618051924, 1618494721, 1618937518, 1619380315, 1619823112, 1620265909, 1620708706, 1621151503, 1621594300, 1622037097, 1622479894, 1622922691, 1623365488, 1623808285, 1624251082, 1624693879, 1625136677, 1625579474, 1626022271, 1626465068, 1626907865, 1627350662, 1627793459, 1628236256, 1628679053, 1629121850, 1629564647, 1630007444, 1630450241, 1630893038, 1631335835, 1631778632, 1632221430, 1632664227, 1633107024, 1633549821, 1633992618, 1634435415, 1634878212, 1635321009, 1635763806, 1636206603, 1636649400, 1637092197, 1637534994, 1637977791, 1638420588, 1638863385, 1639306182, 1639748980, 1640191777, 1640634574, 1641077371, 1641520168, 1641962965, 1642405762, 1642848559, 1643291356, 1643734153, 1644176950, 1644619747, 1645062544, 1645505341, 1645948138, 1646390935, 1646833733, 1647276530, 1647719327, 1648162124, 1648604921, 1649047718, 1649490515, 1649933312, 1650376109, 1650818906, 1651261703, 1651704500, 1652147297, 1652590094, 1653032891, 1653475688, 1653918486, 1654361283, 1654804080, 1655246877, 1655689674, 1656132471, 1656575268, 1657018065, 1657460862, 1657903659, 1658346456, 1658789253, 1659232050, 1659674847, 1660117644, 1660560441, 1661003239, 1661446036, 1661888833, 1662331630, 1662774427, 1663217224, 1663660021, 1664102818, 1664545615, 1664988412, 1665431209, 1665874006, 1666316803, 1666759600, 1667202397, 1667645194, 1668087992, 1668530789, 1668973586, 1669416383, 1669859180, 1670301977, 1670744774, 1671187571, 1671630368, 1672073165, 1672515962, 1672958759, 1673401556, 1673844353, 1674287150, 1674729947, 1675172745, 1675615542, 1676058339, 1676501136, 1676943933, 1677386730, 1677829527, 1678272324, 1678715121, 1679157918, 1679600715, 1680043512, 1680486309, 1680929106, 1681371903, 1681814700, 1682257497, 1682700295, 1683143092, 1683585889, 1684028686, 1684471483, 1684914280, 1685357077, 1685799874, 1686242671, 1686685468, 1687128265, 1687571062, 1688013859, 1688456656, 1688899453, 1689342250, 1689785048, 1690227845, 1690670642, 1691113439, 1691556236, 1691999033, 1692441830, 1692884627, 1693327424, 1693770221, 1694213018, 1694655815, 1695098612, 1695541409, 1695984206, 1696427003, 1696869801, 1697312598, 1697755395, 1698198192, 1698640989, 1699083786, 1699526583, 1699969380, 1700412177, 1700854974, 1701297771, 1701740568, 1702183365, 1702626162, 1703068959, 1703511756, 1703954554, 1704397351, 1704840148, 1705282945, 1705725742, 1706168539, 1706611336, 1707054133, 1707496930, 1707939727, 1708382524, 1708825321, 1709268118, 1709710915, 1710153712, 1710596509, 1711039307, 1711482104, 1711924901],
        "returns_pct": [0.0, 1.02, 4.4, 4.94, 6.86, 6.99, 9.95, 12.15, 10.32, 11.62, 12.39, 13.31, 13.96, 12.15, 16.01, 18.36, 18.83, 18.6, 21.03, 22.52, 23.04, 23.92, 25.73, 25.69, 26.71, 23.57, 18.49, 20.46, 18.17, 16.54, 13.75, 20.5, 22.11, 23.9, 25.03, 24.53, 26.96, 26.93, 28.29, 26.77, 29.45, 28.68, 24.86, 24.54, 21.71, 25.08, 21.2, 24.46, 23.2, 26.61, 28.76, 29.34, 26.37, 25.71, 27.13, 26.16, 28.54, 29.31, 30.97, 32.07, 34.79, 35.87, 36.31, 37.55, 36.67, 38.3, 37.0, 38.2, 40.16, 42.62, 43.42, 44.06, 45.33, 48.17, 48.81, 50.62, 50.3, 50.3, 53.39, 54.81, 58.23, 59.43, 49.93, 39.86, 42.59, 32.31, 15.9, 15.54, 28.45, 28.57, 24.01, 34.5, 42.54, 36.89, 42.12, 43.1, 47.52, 52.54, 50.17, 52.59, 53.26, 54.59, 59.13, 57.25, 63.25, 63.79, 63.55, 69.47, 70.0, 72.32, 72.65, 75.8, 72.43, 73.43, 80.52, 80.48, 81.95, 84.02, 88.76, 92.22, 86.33, 79.0, 80.52, 74.24, 75.45, 78.11, 81.81, 90.97, 86.12, 83.56, 82.69, 74.15, 86.69, 86.63, 89.19, 86.86, 90.77, 93.44, 94.29, 92.56, 97.42, 96.4, 100.1, 99.77, 100.0, 98.7, 97.06, 104.72, 104.1, 104.41, 106.78, 107.12, 105.33, 100.81, 96.28, 93.08, 97.31, 101.27, 98.17, 98.09, 99.28, 103.24, 113.17, 114.43, 115.87, 118.15, 113.42, 109.03, 108.9, 109.16, 108.59, 111.27, 110.06, 110.97, 114.19, 116.04, 116.1, 120.38, 125.19, 128.4, 130.5, 129.29, 130.54, 134.62, 132.78, 134.46, 134.04, 136.2, 135.14, 136.5, 140.49, 142.01, 138.28, 138.47, 131.48, 135.88, 130.66, 129.5, 129.18, 136.32, 139.81, 139.75, 140.78, 146.43, 144.32, 147.52, 151.22, 144.37, 140.97, 143.34, 148.68, 146.01, 146.0, 151.0, 149.21, 141.7, 144.15, 141.46, 126.56, 120.34, 135.4, 135.21, 130.08, 134.74, 127.42, 127.24, 126.16, 114.11, 110.95, 128.74, 137.75, 144.77, 139.23, 133.16, 129.72, 124.69, 118.12, 111.34, 124.09, 105.76, 108.12, 102.97, 102.41, 115.04, 115.63, 109.16, 92.91, 87.64, 98.36, 93.38, 91.14, 107.75, 104.86, 112.68, 122.99, 143.77, 152.69, 136.2, 148.89, 145.94, 145.02, 128.29, 118.86, 136.52, 135.6, 139.13, 120.6, 111.37, 102.54, 90.98, 83.04, 87.4, 98.26, 101.44, 86.74, 74.49, 88.97, 87.02, 86.97, 91.92, 86.49, 82.92, 73.14, 63.7, 58.98, 57.59, 50.63, 60.3, 68.56, 74.01, 81.68, 95.65, 112.29, 113.17, 125.08, 119.56, 109.23, 105.32, 106.16, 96.07, 105.67, 113.63, 111.39, 126.57, 120.02, 117.35, 120.73, 118.23, 119.78, 116.54, 120.5, 119.11, 125.62, 124.26, 133.37, 139.97, 142.5, 150.11, 154.05, 150.79, 151.04, 156.88, 155.36, 160.0, 163.62, 162.8, 164.78, 159.94, 157.11, 155.0, 150.42, 154.52, 157.95, 160.44, 155.3, 154.53, 154.81, 148.87, 145.97, 142.72, 149.04, 149.03, 145.61, 144.11, 136.59, 147.97, 152.58, 154.84, 160.47, 163.21, 163.26, 163.13, 166.27, 172.63, 175.99, 175.32, 175.87, 169.67, 176.24, 174.88, 186.95, 189.17, 188.8, 189.19, 188.22, 191.27, 191.19, 194.12, 201.56, 202.33, 204.05, 196.75, 197.88, 197.23, 198.97],
    },
    "Ex18 — Chronos Fine-tuned": {
        "color": "#ffa15a",
        "timestamps": [1546318800, 1546761597, 1547204394, 1547647191, 1548089988, 1548532785, 1548975582, 1549418379, 1549861176, 1550303973, 1550746770, 1551189567, 1551632364, 1552075161, 1552517958, 1552960755, 1553403553, 1553846350, 1554289147, 1554731944, 1555174741, 1555617538, 1556060335, 1556503132, 1556945929, 1557388726, 1557831523, 1558274320, 1558717117, 1559159914, 1559602711, 1560045508, 1560488306, 1560931103, 1561373900, 1561816697, 1562259494, 1562702291, 1563145088, 1563587885, 1564030682, 1564473479, 1564916276, 1565359073, 1565801870, 1566244667, 1566687464, 1567130261, 1567573059, 1568015856, 1568458653, 1568901450, 1569344247, 1569787044, 1570229841, 1570672638, 1571115435, 1571558232, 1572001029, 1572443826, 1572886623, 1573329420, 1573772217, 1574215015, 1574657812, 1575100609, 1575543406, 1575986203, 1576429000, 1576871797, 1577314594, 1577757391, 1578200188, 1578642985, 1579085782, 1579528579, 1579971376, 1580414173, 1580856970, 1581299768, 1581742565, 1582185362, 1582628159, 1583070956, 1583513753, 1583956550, 1584399347, 1584842144, 1585284941, 1585727738, 1586170535, 1586613332, 1587056129, 1587498926, 1587941723, 1588384521, 1588827318, 1589270115, 1589712912, 1590155709, 1590598506, 1591041303, 1591484100, 1591926897, 1592369694, 1592812491, 1593255288, 1593698085, 1594140882, 1594583679, 1595026476, 1595469274, 1595912071, 1596354868, 1596797665, 1597240462, 1597683259, 1598126056, 1598568853, 1599011650, 1599454447, 1599897244, 1600340041, 1600782838, 1601225635, 1601668432, 1602111230, 1602554027, 1602996824, 1603439621, 1603882418, 1604325215, 1604768012, 1605210809, 1605653606, 1606096403, 1606539200, 1606981997, 1607424794, 1607867591, 1608310388, 1608753185, 1609195983, 1609638780, 1610081577, 1610524374, 1610967171, 1611409968, 1611852765, 1612295562, 1612738359, 1613181156, 1613623953, 1614066750, 1614509547, 1614952344, 1615395141, 1615837938, 1616280736, 1616723533, 1617166330, 1617609127, 1618051924, 1618494721, 1618937518, 1619380315, 1619823112, 1620265909, 1620708706, 1621151503, 1621594300, 1622037097, 1622479894, 1622922691, 1623365489, 1623808286, 1624251083, 1624693880, 1625136677, 1625579474, 1626022271, 1626465068, 1626907865, 1627350662, 1627793459, 1628236256, 1628679053, 1629121850, 1629564647, 1630007445, 1630450242, 1630893039, 1631335836, 1631778633, 1632221430, 1632664227, 1633107024, 1633549821, 1633992618, 1634435415, 1634878212, 1635321009, 1635763806, 1636206603, 1636649400, 1637092198, 1637534995, 1637977792, 1638420589, 1638863386, 1639306183, 1639748980, 1640191777, 1640634574, 1641077371, 1641520168, 1641962965, 1642405762, 1642848559, 1643291356, 1643734153, 1644176951, 1644619748, 1645062545, 1645505342, 1645948139, 1646390936, 1646833733, 1647276530, 1647719327, 1648162124, 1648604921, 1649047718, 1649490515, 1649933312, 1650376109, 1650818906, 1651261704, 1651704501, 1652147298, 1652590095, 1653032892, 1653475689, 1653918486, 1654361283, 1654804080, 1655246877, 1655689674, 1656132471, 1656575268, 1657018065, 1657460862, 1657903660, 1658346457, 1658789254, 1659232051, 1659674848, 1660117645, 1660560442, 1661003239, 1661446036, 1661888833, 1662331630, 1662774427, 1663217224, 1663660021, 1664102818, 1664545615, 1664988413, 1665431210, 1665874007, 1666316804, 1666759601, 1667202398, 1667645195, 1668087992, 1668530789, 1668973586, 1669416383, 1669859180, 1670301977, 1670744774, 1671187571, 1671630368, 1672073166, 1672515963, 1672958760, 1673401557, 1673844354, 1674287151, 1674729948, 1675172745, 1675615542, 1676058339, 1676501136, 1676943933, 1677386730, 1677829527, 1678272324, 1678715121, 1679157919, 1679600716, 1680043513, 1680486310, 1680929107, 1681371904, 1681814701, 1682257498, 1682700295, 1683143092, 1683585889, 1684028686, 1684471483, 1684914280, 1685357077, 1685799875, 1686242672, 1686685469, 1687128266, 1687571063, 1688013860, 1688456657, 1688899454, 1689342251, 1689785048, 1690227845, 1690670642, 1691113439, 1691556236, 1691999033, 1692441830, 1692884628, 1693327425, 1693770222, 1694213019, 1694655816, 1695098613, 1695541410, 1695984207, 1696427004, 1696869801, 1697312598, 1697755395, 1698198192, 1698640989, 1699083786, 1699526583, 1699969381, 1700412178, 1700854975, 1701297772, 1701740569, 1702183366, 1702626163, 1703068960, 1703511757, 1703954554, 1704397351, 1704840148, 1705282945, 1705725742, 1706168539, 1706611336, 1707054134, 1707496931, 1707939728, 1708382525, 1708825322, 1709268119, 1709710916, 1710153713, 1710596510, 1711039307, 1711482104, 1711924901],
        "returns_pct": [0.0, 2.89, 6.46, 7.34, 9.04, 9.15, 10.99, 12.89, 11.35, 13.21, 13.72, 14.53, 15.06, 13.19, 16.64, 17.92, 18.0, 17.87, 20.73, 22.19, 22.91, 23.96, 25.84, 25.86, 26.76, 23.63, 18.4, 20.44, 17.87, 16.27, 13.11, 20.09, 21.74, 23.61, 24.8, 24.2, 26.71, 26.06, 27.29, 26.05, 28.49, 28.65, 25.38, 25.06, 22.2, 25.53, 22.04, 25.33, 24.05, 27.34, 29.73, 30.57, 28.31, 27.91, 29.06, 28.47, 31.81, 32.73, 35.42, 36.12, 40.84, 42.52, 43.36, 44.94, 43.37, 45.51, 43.77, 45.71, 48.62, 51.1, 52.82, 54.95, 57.14, 60.47, 62.15, 63.89, 64.01, 65.08, 70.08, 69.58, 73.39, 75.77, 65.76, 52.67, 56.02, 43.54, 23.7, 20.24, 35.43, 35.43, 29.0, 37.39, 59.76, 56.6, 61.67, 54.54, 58.56, 62.87, 62.94, 65.53, 63.55, 67.03, 68.38, 73.2, 77.15, 80.89, 82.24, 94.19, 96.2, 98.58, 101.21, 104.47, 102.44, 104.0, 108.95, 108.25, 111.15, 112.33, 117.69, 120.54, 115.33, 109.91, 111.85, 105.52, 106.42, 107.62, 111.71, 124.88, 117.67, 113.7, 114.56, 102.79, 118.88, 116.4, 118.1, 115.86, 120.11, 124.58, 125.71, 123.01, 130.53, 129.96, 135.4, 133.96, 135.08, 134.31, 132.45, 138.61, 136.12, 137.5, 141.17, 143.39, 141.93, 137.97, 133.47, 130.63, 136.22, 141.5, 137.9, 138.01, 140.19, 144.16, 154.22, 155.25, 156.36, 158.93, 155.62, 151.03, 150.8, 151.03, 150.89, 153.75, 153.22, 154.52, 157.65, 159.43, 158.51, 163.53, 168.2, 172.04, 175.01, 171.82, 173.62, 178.63, 174.13, 175.82, 175.24, 176.55, 174.62, 177.46, 182.56, 183.75, 180.44, 180.98, 173.1, 178.07, 171.63, 170.3, 170.14, 178.49, 185.03, 189.85, 196.02, 205.73, 196.97, 201.0, 208.96, 199.89, 198.9, 197.13, 205.33, 197.28, 198.55, 211.34, 207.95, 204.76, 207.26, 204.31, 188.05, 181.65, 197.47, 196.46, 190.83, 195.47, 187.11, 187.32, 186.85, 173.28, 170.22, 189.48, 198.72, 206.72, 200.71, 196.13, 193.86, 188.35, 181.77, 173.26, 184.51, 163.91, 162.23, 151.64, 154.35, 167.69, 165.09, 159.5, 142.12, 137.95, 151.75, 149.2, 148.96, 171.44, 170.28, 191.46, 193.17, 216.05, 230.25, 203.32, 223.79, 215.72, 214.84, 184.65, 162.31, 179.52, 169.33, 172.77, 154.28, 144.81, 141.97, 127.89, 116.58, 121.03, 134.08, 137.9, 121.1, 103.73, 119.94, 115.8, 116.55, 124.0, 116.3, 112.44, 99.44, 86.51, 79.24, 77.78, 68.05, 79.19, 89.88, 94.38, 101.26, 117.06, 132.66, 132.49, 145.26, 138.6, 127.69, 123.38, 122.65, 111.06, 122.01, 129.4, 127.04, 143.63, 136.64, 133.63, 137.1, 135.62, 137.88, 134.49, 138.35, 137.4, 144.5, 142.4, 152.26, 158.32, 159.42, 166.41, 170.59, 167.15, 167.6, 172.61, 170.98, 175.69, 180.64, 176.78, 180.46, 174.24, 167.91, 164.3, 159.17, 166.29, 170.44, 174.34, 166.76, 165.24, 165.98, 159.73, 156.35, 156.47, 165.03, 164.92, 158.33, 155.32, 147.92, 158.39, 166.93, 170.01, 176.77, 178.37, 178.47, 178.08, 185.73, 190.09, 190.37, 186.82, 185.25, 176.06, 191.13, 192.89, 219.09, 224.62, 226.89, 230.69, 230.46, 240.37, 236.43, 246.19, 257.82, 271.47, 276.59, 266.37, 264.63, 264.31, 264.39],
    },
}

fig = go.Figure()
for label, data in curves.items():
    dates = [datetime.fromtimestamp(t) for t in data['timestamps']]
    fig.add_trace(go.Scatter(
        x=dates, y=data['returns_pct'],
        mode='lines', name=label,
        line=dict(color=data['color'], width=2),
    ))

fig.update_layout(
    title="Ex18 — Chronos Base vs Ex18 — Chronos Fine-tuned",
    xaxis_title="Date",
    yaxis_title="Cumulative Return %",
    template="plotly_white",
    hovermode="x unified",
)
fig.show()

**References:**

- Ansari, A. F. et al. (2024). "Chronos: Learning the Language of Time Series." [arXiv:2403.07815](https://arxiv.org/abs/2403.07815). — Chronos architecture, MeanScaleUniformBins tokenizer (4,096 tokens), training corpus (~10B observations from weather, energy, retail, economics), zero-shot forecasting.
- Vaswani, A. et al. (2017). "Attention Is All You Need." [arXiv:1706.03762](https://arxiv.org/abs/1706.03762). — Transformer encoder-decoder architecture; self-attention and cross-attention mechanisms.
- Raffel, C. et al. (2019). "Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer." [arXiv:1910.10683](https://arxiv.org/abs/1910.10683). — T5 (Text-to-Text Transfer Transformer); pre-trained on C4 corpus (~750GB); text-to-text framing.
- Brown, T. et al. (2020). "Language Models are Few-Shot Learners." [arXiv:2005.14165](https://arxiv.org/abs/2005.14165). — GPT-3 (175 billion parameters) for size comparison with Chronos-tiny (~8M parameters).
- Bommasani, R. et al. (2021). "On the Opportunities and Risks of Foundation Models." Stanford CRFM. [arXiv:2108.07258](https://arxiv.org/abs/2108.07258). — Formal definition: "A foundation model is any model that is trained on broad data that can be adapted to a wide range of downstream tasks."
- Markowitz, H. (1952). "Portfolio Selection." *The Journal of Finance*, 7(1), 77–91. — Mean-variance optimization framework; covariance matrix as central to portfolio construction.
- Sharpe, W. F. (1966). "Mutual Fund Performance." *The Journal of Business*, 39(1), 119–138. — Original Sharpe ratio (reward-to-variability ratio) definition.
- Sharpe, W. F. (1994). "The Sharpe Ratio." *The Journal of Portfolio Management*, 21(1), 49–58. — Updated Sharpe ratio formulation.
- Pik, J. et al. (2025). *Hands-On AI Trading*. Wiley. pp. ~480–520. — Ex18 Chronos strategy design, lookahead bias warning (p. 426).

## Three Generations — The Full Picture

### Slide 16 — The Progression

Three Generations — The Full Picture

|                  | Ridge (Classical) | CNN (Deep Learning) | Chronos Base | Chronos Fine-Tuned |
|------------------|-------------------|---------------------|--------------|--------------------|
| Sharpe       | 0.212             | 0.649               | 0.727        | 0.846              |
| Net Profit   | +35%              | +145%               | +200%        | +266%              |
| Alpha        | -0.062            | 0.093               | 0.040        | 0.076              |
| Beta         | 1.146             | 0.278               | 1.125        | 1.110              |
| Max Drawdown | 54.7%             | 26.3%               | 41.2%        | 49.5%              |
| Training     | Weekly, per contract | Weekly, 20 epochs × 3 stocks | None | 3 steps |

The trend: Less effort, better results.

---

Here's the full progression. Sharpe ratio: 0.2 → 0.6 → 0.7 → 0.8. Net profit: 35% → 145% → 200% → 266%. Each generation improves.

But look at the training effort line at the bottom. Ridge regression requires manual feature engineering — someone had to decide to use volatility, ATR, and open interest. The CNN requires weekly retraining — 20 epochs times 3 securities times 52 weeks a year. That's roughly 780 training runs over 5 years. Chronos? Zero for the base model. Three gradient steps for fine-tuned.

Less effort, better results. That's the foundation model story.

Now, I want to be nuanced here. The CNN is actually the cleanest alpha generator — alpha 0.093 with beta 0.278. The Chronos models have higher total returns but much of it comes from market exposure — beta above 1.1. So if you care about _pure skill_, the CNN wins. If you care about _total returns with minimal effort_, Chronos wins.


### Slide 17 — Caveats & Honesty

Intellectual Honesty — The Caveats

1. Lookahead bias (Chronos): Model released 2024, likely trained on post-2019 data. Backtest starts 2019. Base model performance may be inflated.

2. Statistical significance: No strategy achieves PSR > 95%. Best is Chronos fine-tuned at 34.6% (Bailey & López de Prado, 2012). These results are suggestive, not definitive.

3. Different universes: Futures (#1) ≠ QQQ stocks (#2) ≠ Top-5 liquid (#3). Apples to oranges — but the cross-domain comparison strengthens the story.

4. Survivorship bias: Universe selection uses current ETF/index constituents, not historical.

5. Small universes: 3 stocks (CNN) and 5 stocks (Chronos). Limited diversification.

---

I want to be honest about what these results don't prove. First, lookahead bias. Chronos was released in 2024 and was trained on massive datasets that likely include financial data from our backtest period. The base model may have 'seen' some of this data during pre-training. This is a real concern.

Second, none of these strategies are statistically significant at the 95% confidence level. The Probabilistic Sharpe Ratio — which asks 'what's the probability this Sharpe is genuinely positive?' — never exceeds 35%. These are suggestive results, not proof.

Third, we're comparing different universes. Futures, QQQ stocks, and top-5-by-dollar-volume are fundamentally different asset pools. You can't directly compare them. But I'd argue the cross-domain comparison actually _strengthens_ the story — Chronos achieves the best risk-adjusted returns on a generic dollar-volume universe with zero feature engineering.

Fourth and fifth: survivorship bias and small universes. We're trading 3-5 mega-cap stocks selected by current metrics. These are limitations of the book exercises, not the techniques.


In [35]:
import plotly.graph_objects as go
from datetime import datetime

# Equity curve data (cumulative return %)
curves = {
    "Ex11 — Ridge Regression": {
        "color": "#ef553b",
        "timestamps": [1546232400, 1546675197, 1547117994, 1547560791, 1548003588, 1548446385, 1548889182, 1549331979, 1549774776, 1550217573, 1550660370, 1551103167, 1551545964, 1551988761, 1552431558, 1552874355, 1553317152, 1553759950, 1554202747, 1554645544, 1555088341, 1555531138, 1555973935, 1556416732, 1556859529, 1557302326, 1557745123, 1558187920, 1558630717, 1559073514, 1559516311, 1559959108, 1560401905, 1560844702, 1561287500, 1561730297, 1562173094, 1562615891, 1563058688, 1563501485, 1563944282, 1564387079, 1564829876, 1565272673, 1565715470, 1566158267, 1566601064, 1567043861, 1567486658, 1567929455, 1568372252, 1568815050, 1569257847, 1569700644, 1570143441, 1570586238, 1571029035, 1571471832, 1571914629, 1572357426, 1572800223, 1573243020, 1573685817, 1574128614, 1574571411, 1575014208, 1575457005, 1575899802, 1576342600, 1576785397, 1577228194, 1577670991, 1578113788, 1578556585, 1578999382, 1579442179, 1579884976, 1580327773, 1580770570, 1581213367, 1581656164, 1582098961, 1582541758, 1582984555, 1583427352, 1583870150, 1584312947, 1584755744, 1585198541, 1585641338, 1586084135, 1586526932, 1586969729, 1587412526, 1587855323, 1588298120, 1588740917, 1589183714, 1589626511, 1590069308, 1590512105, 1590954902, 1591397700, 1591840497, 1592283294, 1592726091, 1593168888, 1593611685, 1594054482, 1594497279, 1594940076, 1595382873, 1595825670, 1596268467, 1596711264, 1597154061, 1597596858, 1598039655, 1598482452, 1598925250, 1599368047, 1599810844, 1600253641, 1600696438, 1601139235, 1601582032, 1602024829, 1602467626, 1602910423, 1603353220, 1603796017, 1604238814, 1604681611, 1605124408, 1605567205, 1606010003, 1606452800, 1606895597, 1607338394, 1607781191, 1608223988, 1608666785, 1609109582, 1609552379, 1609995176, 1610437973, 1610880770, 1611323567, 1611766364, 1612209161, 1612651958, 1613094755, 1613537553, 1613980350, 1614423147, 1614865944, 1615308741, 1615751538, 1616194335, 1616637132, 1617079929, 1617522726, 1617965523, 1618408320, 1618851117, 1619293914, 1619736711, 1620179508, 1620622305, 1621065103, 1621507900, 1621950697, 1622393494, 1622836291, 1623279088, 1623721885, 1624164682, 1624607479, 1625050276, 1625493073, 1625935870, 1626378667, 1626821464, 1627264261, 1627707058, 1628149855, 1628592653, 1629035450, 1629478247, 1629921044, 1630363841, 1630806638, 1631249435, 1631692232, 1632135029, 1632577826, 1633020623, 1633463420, 1633906217, 1634349014, 1634791811, 1635234608, 1635677405, 1636120203, 1636563000, 1637005797, 1637448594, 1637891391, 1638334188, 1638776985, 1639219782, 1639662579, 1640105376, 1640548173, 1640990970, 1641433767, 1641876564, 1642319361, 1642762158, 1643204955, 1643647753, 1644090550, 1644533347, 1644976144, 1645418941, 1645861738, 1646304535, 1646747332, 1647190129, 1647632926, 1648075723, 1648518520, 1648961317, 1649404114, 1649846911, 1650289708, 1650732505, 1651175303, 1651618100, 1652060897, 1652503694, 1652946491, 1653389288, 1653832085, 1654274882, 1654717679, 1655160476, 1655603273, 1656046070, 1656488867, 1656931664, 1657374461, 1657817258, 1658260056, 1658702853, 1659145650, 1659588447, 1660031244, 1660474041, 1660916838, 1661359635, 1661802432, 1662245229, 1662688026, 1663130823, 1663573620, 1664016417, 1664459214, 1664902011, 1665344808, 1665787606, 1666230403, 1666673200, 1667115997, 1667558794, 1668001591, 1668444388, 1668887185, 1669329982, 1669772779, 1670215576, 1670658373, 1671101170, 1671543967, 1671986764, 1672429561, 1672872358, 1673315156, 1673757953, 1674200750, 1674643547, 1675086344, 1675529141, 1675971938, 1676414735, 1676857532, 1677300329, 1677743126, 1678185923, 1678628720, 1679071517, 1679514314, 1679957111, 1680399908, 1680842706, 1681285503, 1681728300, 1682171097, 1682613894, 1683056691, 1683499488, 1683942285, 1684385082, 1684827879, 1685270676, 1685713473, 1686156270, 1686599067, 1687041864, 1687484661, 1687927458, 1688370256, 1688813053, 1689255850, 1689698647, 1690141444, 1690584241, 1691027038, 1691469835, 1691912632, 1692355429, 1692798226, 1693241023, 1693683820, 1694126617, 1694569414, 1695012211, 1695455008, 1695897806, 1696340603, 1696783400, 1697226197, 1697668994, 1698111791, 1698554588, 1698997385, 1699440182, 1699882979, 1700325776, 1700768573, 1701211370, 1701654167, 1702096964, 1702539761, 1702982558, 1703425356, 1703868153, 1704310950, 1704753747, 1705196544, 1705639341, 1706082138, 1706524935, 1706967732, 1707410529, 1707853326, 1708296123, 1708738920, 1709181717, 1709624514, 1710067311, 1710510109, 1710952906, 1711395703, 1711838500],
        "returns_pct": [0.0, 0.53, 2.03, 3.05, 6.59, 6.65, 7.89, 9.17, 8.73, 11.59, 14.28, 16.11, 15.56, 7.65, 8.6, 11.73, 11.78, 12.36, 18.05, 20.47, 21.02, 20.83, 21.93, 22.68, 21.8, 18.04, 12.04, 16.74, 13.97, 10.63, 3.21, 20.13, 22.13, 26.32, 29.37, 29.31, 32.32, 27.26, 35.23, 35.5, 34.9, 34.34, 22.87, 18.33, 17.67, 14.98, 9.87, 13.09, 15.56, 20.38, 23.74, 22.21, 23.21, 23.3, 14.61, 15.35, 22.97, 22.64, 23.24, 26.47, 28.91, 33.13, 34.43, 38.67, 36.12, 39.88, 34.41, 37.99, 41.05, 45.04, 46.24, 49.36, 48.84, 52.14, 54.17, 59.45, 54.98, 49.87, 49.62, 57.6, 65.6, 63.78, 43.68, 15.12, 13.35, -4.53, -14.27, -17.63, -17.68, -17.05, -18.28, -15.49, -15.1, -15.11, -14.63, -14.24, -13.98, -13.11, -14.09, -12.67, -11.01, -10.28, -4.44, -4.44, -7.1, -6.36, -6.27, -4.56, -3.18, -3.05, -2.28, -0.89, -2.73, -2.06, 0.11, 3.13, 2.66, 3.55, 6.08, 7.43, 5.21, 3.0, 4.79, 3.55, 3.47, 4.21, 4.27, 7.69, 7.45, 6.4, 3.78, 0.03, 7.28, 10.46, 13.4, 10.3, 13.62, 13.75, 15.07, 14.72, 15.98, 16.71, 16.73, 17.94, 19.52, 20.67, 18.61, 21.47, 16.64, 16.77, 19.24, 20.75, 21.49, 20.45, 16.37, 18.38, 20.65, 27.19, 31.15, 31.29, 31.94, 33.38, 37.42, 40.55, 42.72, 41.81, 42.01, 42.43, 47.89, 43.85, 38.51, 43.06, 45.53, 47.42, 45.78, 46.27, 39.71, 39.02, 39.75, 41.96, 43.28, 43.21, 40.42, 45.39, 45.58, 46.12, 47.98, 50.35, 47.93, 50.72, 52.64, 52.46, 50.45, 46.89, 46.97, 50.74, 42.39, 44.01, 48.93, 55.4, 57.82, 61.68, 62.76, 67.9, 65.53, 66.1, 64.21, 65.47, 54.48, 56.73, 65.47, 67.3, 65.08, 68.63, 70.96, 69.14, 65.27, 64.44, 50.32, 46.13, 48.93, 49.67, 50.02, 49.0, 43.5, 42.64, 42.71, 32.54, 34.61, 43.37, 43.92, 47.1, 44.89, 42.37, 35.56, 35.0, 27.11, 27.44, 21.02, 15.99, 13.32, 7.85, 7.41, 18.73, 17.19, 16.46, 0.92, -4.75, -4.53, -7.66, -8.81, -6.22, -9.68, -2.35, -2.37, 4.55, 4.81, 4.61, 11.85, 11.3, 5.62, -2.16, -8.16, -5.42, -9.59, -12.75, -13.96, -16.0, -10.17, -16.34, -15.19, -10.75, -2.84, 2.77, -1.62, 1.29, 6.39, 5.77, 8.61, 6.09, 9.8, 4.42, 3.78, -0.17, -0.3, -0.31, -0.56, 0.05, 1.8, 1.05, 1.63, 2.89, 4.42, 3.34, 5.08, 3.86, -3.05, -4.58, -2.2, -14.95, -14.98, -14.9, -14.99, -11.98, -11.51, -10.63, -10.35, -10.76, -10.89, -11.71, -12.14, -14.25, -13.13, -12.86, -12.71, -8.92, -9.74, -6.41, -2.05, -2.26, -1.88, 1.17, -2.64, 0.86, 5.93, 6.47, 8.94, 6.5, 3.84, 4.2, -5.05, -2.8, -1.5, 1.78, -0.2, 0.21, 2.19, 1.62, -2.88, -8.46, -3.67, -2.86, -2.9, -8.53, -13.74, -3.37, -1.6, 0.06, 3.69, 6.58, 8.11, 13.69, 14.04, 22.04, 23.21, 23.26, 24.47, 20.78, 22.81, 22.3, 21.7, 28.32, 29.68, 33.59, 35.27, 30.62, 34.37, 39.23, 37.79, 35.77, 36.12, 34.55, 35.33, 34.8, 37.0],
    },
    "Ex14 — Temporal CNN": {
        "color": "#636efa",
        "timestamps": [1546232400, 1546675197, 1547117994, 1547560791, 1548003588, 1548446385, 1548889182, 1549331979, 1549774776, 1550217573, 1550660370, 1551103167, 1551545964, 1551988761, 1552431558, 1552874355, 1553317152, 1553759950, 1554202747, 1554645544, 1555088341, 1555531138, 1555973935, 1556416732, 1556859529, 1557302326, 1557745123, 1558187920, 1558630717, 1559073514, 1559516311, 1559959108, 1560401905, 1560844702, 1561287500, 1561730297, 1562173094, 1562615891, 1563058688, 1563501485, 1563944282, 1564387079, 1564829876, 1565272673, 1565715470, 1566158267, 1566601064, 1567043861, 1567486658, 1567929455, 1568372253, 1568815050, 1569257847, 1569700644, 1570143441, 1570586238, 1571029035, 1571471832, 1571914629, 1572357426, 1572800223, 1573243020, 1573685817, 1574128614, 1574571411, 1575014208, 1575457005, 1575899803, 1576342600, 1576785397, 1577228194, 1577670991, 1578113788, 1578556585, 1578999382, 1579442179, 1579884976, 1580327773, 1580770570, 1581213367, 1581656164, 1582098961, 1582541758, 1582984556, 1583427353, 1583870150, 1584312947, 1584755744, 1585198541, 1585641338, 1586084135, 1586526932, 1586969729, 1587412526, 1587855323, 1588298120, 1588740917, 1589183714, 1589626511, 1590069308, 1590512106, 1590954903, 1591397700, 1591840497, 1592283294, 1592726091, 1593168888, 1593611685, 1594054482, 1594497279, 1594940076, 1595382873, 1595825670, 1596268467, 1596711264, 1597154061, 1597596859, 1598039656, 1598482453, 1598925250, 1599368047, 1599810844, 1600253641, 1600696438, 1601139235, 1601582032, 1602024829, 1602467626, 1602910423, 1603353220, 1603796017, 1604238814, 1604681611, 1605124409, 1605567206, 1606010003, 1606452800, 1606895597, 1607338394, 1607781191, 1608223988, 1608666785, 1609109582, 1609552379, 1609995176, 1610437973, 1610880770, 1611323567, 1611766364, 1612209162, 1612651959, 1613094756, 1613537553, 1613980350, 1614423147, 1614865944, 1615308741, 1615751538, 1616194335, 1616637132, 1617079929, 1617522726, 1617965523, 1618408320, 1618851117, 1619293914, 1619736712, 1620179509, 1620622306, 1621065103, 1621507900, 1621950697, 1622393494, 1622836291, 1623279088, 1623721885, 1624164682, 1624607479, 1625050276, 1625493073, 1625935870, 1626378667, 1626821465, 1627264262, 1627707059, 1628149856, 1628592653, 1629035450, 1629478247, 1629921044, 1630363841, 1630806638, 1631249435, 1631692232, 1632135029, 1632577826, 1633020623, 1633463420, 1633906217, 1634349015, 1634791812, 1635234609, 1635677406, 1636120203, 1636563000, 1637005797, 1637448594, 1637891391, 1638334188, 1638776985, 1639219782, 1639662579, 1640105376, 1640548173, 1640990970, 1641433768, 1641876565, 1642319362, 1642762159, 1643204956, 1643647753, 1644090550, 1644533347, 1644976144, 1645418941, 1645861738, 1646304535, 1646747332, 1647190129, 1647632926, 1648075723, 1648518520, 1648961318, 1649404115, 1649846912, 1650289709, 1650732506, 1651175303, 1651618100, 1652060897, 1652503694, 1652946491, 1653389288, 1653832085, 1654274882, 1654717679, 1655160476, 1655603273, 1656046071, 1656488868, 1656931665, 1657374462, 1657817259, 1658260056, 1658702853, 1659145650, 1659588447, 1660031244, 1660474041, 1660916838, 1661359635, 1661802432, 1662245229, 1662688026, 1663130823, 1663573621, 1664016418, 1664459215, 1664902012, 1665344809, 1665787606, 1666230403, 1666673200, 1667115997, 1667558794, 1668001591, 1668444388, 1668887185, 1669329982, 1669772779, 1670215576, 1670658374, 1671101171, 1671543968, 1671986765, 1672429562, 1672872359, 1673315156, 1673757953, 1674200750, 1674643547, 1675086344, 1675529141, 1675971938, 1676414735, 1676857532, 1677300329, 1677743126, 1678185924, 1678628721, 1679071518, 1679514315, 1679957112, 1680399909, 1680842706, 1681285503, 1681728300, 1682171097, 1682613894, 1683056691, 1683499488, 1683942285, 1684385082, 1684827879, 1685270677, 1685713474, 1686156271, 1686599068, 1687041865, 1687484662, 1687927459, 1688370256, 1688813053, 1689255850, 1689698647, 1690141444, 1690584241, 1691027038, 1691469835, 1691912632, 1692355429, 1692798227, 1693241024, 1693683821, 1694126618, 1694569415, 1695012212, 1695455009, 1695897806, 1696340603, 1696783400, 1697226197, 1697668994, 1698111791, 1698554588, 1698997385, 1699440182, 1699882980, 1700325777, 1700768574, 1701211371, 1701654168, 1702096965, 1702539762, 1702982559, 1703425356, 1703868153, 1704310950, 1704753747, 1705196544, 1705639341, 1706082138, 1706524935, 1706967732, 1707410530, 1707853327, 1708296124, 1708738921, 1709181718, 1709624515, 1710067312, 1710510109, 1710952906, 1711395703, 1711838500],
        "returns_pct": [0.0, -1.33, 0.83, -0.1, -1.06, -0.28, 2.11, 3.04, 3.0, 3.32, 3.1, 1.96, 3.35, 3.47, 4.36, 6.07, 6.28, 5.82, 4.73, 6.3, 6.18, 6.15, 7.24, 10.14, 12.25, 12.36, 13.52, 14.94, 17.03, 17.03, 14.26, 14.08, 15.66, 15.9, 16.4, 17.74, 18.19, 18.99, 20.41, 21.78, 20.85, 19.38, 18.2, 13.85, 13.57, 13.07, 13.23, 13.38, 14.79, 16.69, 15.89, 14.99, 15.05, 13.39, 12.42, 12.87, 12.85, 13.42, 15.18, 13.45, 15.94, 16.52, 18.32, 19.31, 18.17, 20.17, 18.98, 21.11, 21.61, 21.53, 21.83, 19.97, 20.26, 22.37, 24.64, 26.02, 26.43, 26.95, 33.85, 31.93, 33.52, 30.3, 32.59, 41.52, 36.74, 57.01, 52.44, 32.77, 30.77, 30.35, 32.86, 37.66, 45.06, 44.7, 45.93, 48.89, 50.21, 50.55, 49.28, 50.03, 47.46, 48.09, 47.36, 56.45, 49.96, 51.39, 52.52, 57.45, 63.6, 68.45, 64.44, 67.32, 66.55, 76.11, 82.9, 82.69, 81.48, 86.26, 93.11, 94.77, 86.51, 78.11, 75.99, 76.01, 85.92, 92.96, 87.05, 95.77, 96.26, 94.6, 92.35, 83.46, 86.09, 79.36, 79.24, 75.64, 78.57, 75.73, 74.9, 73.23, 79.91, 82.46, 81.93, 83.64, 75.83, 77.82, 75.84, 76.81, 73.49, 80.33, 82.97, 81.12, 80.15, 73.91, 69.18, 67.1, 68.43, 70.02, 68.62, 69.94, 69.28, 69.41, 67.18, 66.04, 65.46, 65.66, 66.22, 61.62, 61.36, 61.27, 61.14, 63.15, 61.34, 61.74, 63.24, 66.33, 68.2, 68.1, 70.37, 74.31, 79.28, 78.4, 78.34, 82.34, 82.26, 82.25, 83.27, 85.9, 86.96, 88.29, 93.24, 93.77, 93.86, 91.09, 86.51, 85.06, 82.25, 82.13, 80.89, 78.55, 75.64, 75.55, 75.18, 77.89, 78.57, 77.42, 85.69, 86.57, 87.82, 88.39, 97.74, 100.4, 105.72, 109.3, 110.66, 101.83, 99.26, 98.62, 98.65, 105.1, 106.91, 107.16, 106.43, 109.02, 106.13, 108.76, 113.84, 109.38, 112.8, 129.04, 122.3, 120.24, 118.37, 117.71, 118.56, 120.47, 116.59, 118.36, 116.36, 118.48, 114.84, 106.39, 101.9, 98.98, 102.6, 100.22, 109.06, 111.08, 123.57, 129.66, 129.26, 138.38, 131.82, 130.7, 133.94, 127.95, 124.86, 126.4, 118.51, 116.54, 113.17, 112.91, 114.35, 116.46, 116.34, 118.79, 120.78, 128.79, 143.88, 137.58, 140.01, 138.65, 147.07, 151.79, 134.57, 136.98, 151.89, 153.38, 155.29, 145.44, 152.91, 153.7, 154.67, 155.95, 156.87, 154.6, 159.16, 152.74, 147.47, 145.25, 150.56, 149.47, 159.33, 159.45, 158.71, 159.6, 156.71, 163.1, 153.87, 155.24, 162.57, 159.28, 153.75, 161.81, 162.03, 164.21, 169.47, 168.78, 180.48, 183.52, 180.27, 180.4, 181.62, 184.17, 180.54, 172.17, 170.97, 174.28, 174.69, 172.45, 172.66, 168.42, 168.16, 167.26, 171.02, 161.14, 162.08, 162.08, 163.31, 164.41, 157.36, 155.74, 157.92, 158.29, 150.64, 156.14, 154.81, 154.74, 164.54, 159.26, 162.43, 159.29, 160.96, 161.04, 161.04, 149.22, 144.86, 144.54, 141.85, 140.37, 139.83, 139.79, 143.23, 136.52, 133.55, 133.27, 133.75, 128.86, 127.08, 122.78, 125.16, 127.74, 126.32, 122.22, 122.34, 121.4, 119.4, 131.99, 134.2, 132.67, 134.97, 134.95, 138.76, 150.02, 144.45],
    },
    "Ex18 — Chronos Base": {
        "color": "#00cc96",
        "timestamps": [1546318800, 1546761597, 1547204394, 1547647191, 1548089988, 1548532785, 1548975582, 1549418379, 1549861176, 1550303973, 1550746770, 1551189567, 1551632364, 1552075161, 1552517958, 1552960755, 1553403552, 1553846350, 1554289147, 1554731944, 1555174741, 1555617538, 1556060335, 1556503132, 1556945929, 1557388726, 1557831523, 1558274320, 1558717117, 1559159914, 1559602711, 1560045508, 1560488305, 1560931103, 1561373900, 1561816697, 1562259494, 1562702291, 1563145088, 1563587885, 1564030682, 1564473479, 1564916276, 1565359073, 1565801870, 1566244667, 1566687464, 1567130261, 1567573058, 1568015856, 1568458653, 1568901450, 1569344247, 1569787044, 1570229841, 1570672638, 1571115435, 1571558232, 1572001029, 1572443826, 1572886623, 1573329420, 1573772217, 1574215014, 1574657811, 1575100609, 1575543406, 1575986203, 1576429000, 1576871797, 1577314594, 1577757391, 1578200188, 1578642985, 1579085782, 1579528579, 1579971376, 1580414173, 1580856970, 1581299767, 1581742564, 1582185362, 1582628159, 1583070956, 1583513753, 1583956550, 1584399347, 1584842144, 1585284941, 1585727738, 1586170535, 1586613332, 1587056129, 1587498926, 1587941723, 1588384520, 1588827317, 1589270115, 1589712912, 1590155709, 1590598506, 1591041303, 1591484100, 1591926897, 1592369694, 1592812491, 1593255288, 1593698085, 1594140882, 1594583679, 1595026476, 1595469273, 1595912070, 1596354867, 1596797665, 1597240462, 1597683259, 1598126056, 1598568853, 1599011650, 1599454447, 1599897244, 1600340041, 1600782838, 1601225635, 1601668432, 1602111229, 1602554026, 1602996823, 1603439620, 1603882418, 1604325215, 1604768012, 1605210809, 1605653606, 1606096403, 1606539200, 1606981997, 1607424794, 1607867591, 1608310388, 1608753185, 1609195982, 1609638779, 1610081576, 1610524373, 1610967171, 1611409968, 1611852765, 1612295562, 1612738359, 1613181156, 1613623953, 1614066750, 1614509547, 1614952344, 1615395141, 1615837938, 1616280735, 1616723532, 1617166329, 1617609126, 1618051924, 1618494721, 1618937518, 1619380315, 1619823112, 1620265909, 1620708706, 1621151503, 1621594300, 1622037097, 1622479894, 1622922691, 1623365488, 1623808285, 1624251082, 1624693879, 1625136677, 1625579474, 1626022271, 1626465068, 1626907865, 1627350662, 1627793459, 1628236256, 1628679053, 1629121850, 1629564647, 1630007444, 1630450241, 1630893038, 1631335835, 1631778632, 1632221430, 1632664227, 1633107024, 1633549821, 1633992618, 1634435415, 1634878212, 1635321009, 1635763806, 1636206603, 1636649400, 1637092197, 1637534994, 1637977791, 1638420588, 1638863385, 1639306182, 1639748980, 1640191777, 1640634574, 1641077371, 1641520168, 1641962965, 1642405762, 1642848559, 1643291356, 1643734153, 1644176950, 1644619747, 1645062544, 1645505341, 1645948138, 1646390935, 1646833733, 1647276530, 1647719327, 1648162124, 1648604921, 1649047718, 1649490515, 1649933312, 1650376109, 1650818906, 1651261703, 1651704500, 1652147297, 1652590094, 1653032891, 1653475688, 1653918486, 1654361283, 1654804080, 1655246877, 1655689674, 1656132471, 1656575268, 1657018065, 1657460862, 1657903659, 1658346456, 1658789253, 1659232050, 1659674847, 1660117644, 1660560441, 1661003239, 1661446036, 1661888833, 1662331630, 1662774427, 1663217224, 1663660021, 1664102818, 1664545615, 1664988412, 1665431209, 1665874006, 1666316803, 1666759600, 1667202397, 1667645194, 1668087992, 1668530789, 1668973586, 1669416383, 1669859180, 1670301977, 1670744774, 1671187571, 1671630368, 1672073165, 1672515962, 1672958759, 1673401556, 1673844353, 1674287150, 1674729947, 1675172745, 1675615542, 1676058339, 1676501136, 1676943933, 1677386730, 1677829527, 1678272324, 1678715121, 1679157918, 1679600715, 1680043512, 1680486309, 1680929106, 1681371903, 1681814700, 1682257497, 1682700295, 1683143092, 1683585889, 1684028686, 1684471483, 1684914280, 1685357077, 1685799874, 1686242671, 1686685468, 1687128265, 1687571062, 1688013859, 1688456656, 1688899453, 1689342250, 1689785048, 1690227845, 1690670642, 1691113439, 1691556236, 1691999033, 1692441830, 1692884627, 1693327424, 1693770221, 1694213018, 1694655815, 1695098612, 1695541409, 1695984206, 1696427003, 1696869801, 1697312598, 1697755395, 1698198192, 1698640989, 1699083786, 1699526583, 1699969380, 1700412177, 1700854974, 1701297771, 1701740568, 1702183365, 1702626162, 1703068959, 1703511756, 1703954554, 1704397351, 1704840148, 1705282945, 1705725742, 1706168539, 1706611336, 1707054133, 1707496930, 1707939727, 1708382524, 1708825321, 1709268118, 1709710915, 1710153712, 1710596509, 1711039307, 1711482104, 1711924901],
        "returns_pct": [0.0, 1.02, 4.4, 4.94, 6.86, 6.99, 9.95, 12.15, 10.32, 11.62, 12.39, 13.31, 13.96, 12.15, 16.01, 18.36, 18.83, 18.6, 21.03, 22.52, 23.04, 23.92, 25.73, 25.69, 26.71, 23.57, 18.49, 20.46, 18.17, 16.54, 13.75, 20.5, 22.11, 23.9, 25.03, 24.53, 26.96, 26.93, 28.29, 26.77, 29.45, 28.68, 24.86, 24.54, 21.71, 25.08, 21.2, 24.46, 23.2, 26.61, 28.76, 29.34, 26.37, 25.71, 27.13, 26.16, 28.54, 29.31, 30.97, 32.07, 34.79, 35.87, 36.31, 37.55, 36.67, 38.3, 37.0, 38.2, 40.16, 42.62, 43.42, 44.06, 45.33, 48.17, 48.81, 50.62, 50.3, 50.3, 53.39, 54.81, 58.23, 59.43, 49.93, 39.86, 42.59, 32.31, 15.9, 15.54, 28.45, 28.57, 24.01, 34.5, 42.54, 36.89, 42.12, 43.1, 47.52, 52.54, 50.17, 52.59, 53.26, 54.59, 59.13, 57.25, 63.25, 63.79, 63.55, 69.47, 70.0, 72.32, 72.65, 75.8, 72.43, 73.43, 80.52, 80.48, 81.95, 84.02, 88.76, 92.22, 86.33, 79.0, 80.52, 74.24, 75.45, 78.11, 81.81, 90.97, 86.12, 83.56, 82.69, 74.15, 86.69, 86.63, 89.19, 86.86, 90.77, 93.44, 94.29, 92.56, 97.42, 96.4, 100.1, 99.77, 100.0, 98.7, 97.06, 104.72, 104.1, 104.41, 106.78, 107.12, 105.33, 100.81, 96.28, 93.08, 97.31, 101.27, 98.17, 98.09, 99.28, 103.24, 113.17, 114.43, 115.87, 118.15, 113.42, 109.03, 108.9, 109.16, 108.59, 111.27, 110.06, 110.97, 114.19, 116.04, 116.1, 120.38, 125.19, 128.4, 130.5, 129.29, 130.54, 134.62, 132.78, 134.46, 134.04, 136.2, 135.14, 136.5, 140.49, 142.01, 138.28, 138.47, 131.48, 135.88, 130.66, 129.5, 129.18, 136.32, 139.81, 139.75, 140.78, 146.43, 144.32, 147.52, 151.22, 144.37, 140.97, 143.34, 148.68, 146.01, 146.0, 151.0, 149.21, 141.7, 144.15, 141.46, 126.56, 120.34, 135.4, 135.21, 130.08, 134.74, 127.42, 127.24, 126.16, 114.11, 110.95, 128.74, 137.75, 144.77, 139.23, 133.16, 129.72, 124.69, 118.12, 111.34, 124.09, 105.76, 108.12, 102.97, 102.41, 115.04, 115.63, 109.16, 92.91, 87.64, 98.36, 93.38, 91.14, 107.75, 104.86, 112.68, 122.99, 143.77, 152.69, 136.2, 148.89, 145.94, 145.02, 128.29, 118.86, 136.52, 135.6, 139.13, 120.6, 111.37, 102.54, 90.98, 83.04, 87.4, 98.26, 101.44, 86.74, 74.49, 88.97, 87.02, 86.97, 91.92, 86.49, 82.92, 73.14, 63.7, 58.98, 57.59, 50.63, 60.3, 68.56, 74.01, 81.68, 95.65, 112.29, 113.17, 125.08, 119.56, 109.23, 105.32, 106.16, 96.07, 105.67, 113.63, 111.39, 126.57, 120.02, 117.35, 120.73, 118.23, 119.78, 116.54, 120.5, 119.11, 125.62, 124.26, 133.37, 139.97, 142.5, 150.11, 154.05, 150.79, 151.04, 156.88, 155.36, 160.0, 163.62, 162.8, 164.78, 159.94, 157.11, 155.0, 150.42, 154.52, 157.95, 160.44, 155.3, 154.53, 154.81, 148.87, 145.97, 142.72, 149.04, 149.03, 145.61, 144.11, 136.59, 147.97, 152.58, 154.84, 160.47, 163.21, 163.26, 163.13, 166.27, 172.63, 175.99, 175.32, 175.87, 169.67, 176.24, 174.88, 186.95, 189.17, 188.8, 189.19, 188.22, 191.27, 191.19, 194.12, 201.56, 202.33, 204.05, 196.75, 197.88, 197.23, 198.97],
    },
    "Ex18 — Chronos Fine-tuned": {
        "color": "#ffa15a",
        "timestamps": [1546318800, 1546761597, 1547204394, 1547647191, 1548089988, 1548532785, 1548975582, 1549418379, 1549861176, 1550303973, 1550746770, 1551189567, 1551632364, 1552075161, 1552517958, 1552960755, 1553403553, 1553846350, 1554289147, 1554731944, 1555174741, 1555617538, 1556060335, 1556503132, 1556945929, 1557388726, 1557831523, 1558274320, 1558717117, 1559159914, 1559602711, 1560045508, 1560488306, 1560931103, 1561373900, 1561816697, 1562259494, 1562702291, 1563145088, 1563587885, 1564030682, 1564473479, 1564916276, 1565359073, 1565801870, 1566244667, 1566687464, 1567130261, 1567573059, 1568015856, 1568458653, 1568901450, 1569344247, 1569787044, 1570229841, 1570672638, 1571115435, 1571558232, 1572001029, 1572443826, 1572886623, 1573329420, 1573772217, 1574215015, 1574657812, 1575100609, 1575543406, 1575986203, 1576429000, 1576871797, 1577314594, 1577757391, 1578200188, 1578642985, 1579085782, 1579528579, 1579971376, 1580414173, 1580856970, 1581299768, 1581742565, 1582185362, 1582628159, 1583070956, 1583513753, 1583956550, 1584399347, 1584842144, 1585284941, 1585727738, 1586170535, 1586613332, 1587056129, 1587498926, 1587941723, 1588384521, 1588827318, 1589270115, 1589712912, 1590155709, 1590598506, 1591041303, 1591484100, 1591926897, 1592369694, 1592812491, 1593255288, 1593698085, 1594140882, 1594583679, 1595026476, 1595469274, 1595912071, 1596354868, 1596797665, 1597240462, 1597683259, 1598126056, 1598568853, 1599011650, 1599454447, 1599897244, 1600340041, 1600782838, 1601225635, 1601668432, 1602111230, 1602554027, 1602996824, 1603439621, 1603882418, 1604325215, 1604768012, 1605210809, 1605653606, 1606096403, 1606539200, 1606981997, 1607424794, 1607867591, 1608310388, 1608753185, 1609195983, 1609638780, 1610081577, 1610524374, 1610967171, 1611409968, 1611852765, 1612295562, 1612738359, 1613181156, 1613623953, 1614066750, 1614509547, 1614952344, 1615395141, 1615837938, 1616280736, 1616723533, 1617166330, 1617609127, 1618051924, 1618494721, 1618937518, 1619380315, 1619823112, 1620265909, 1620708706, 1621151503, 1621594300, 1622037097, 1622479894, 1622922691, 1623365489, 1623808286, 1624251083, 1624693880, 1625136677, 1625579474, 1626022271, 1626465068, 1626907865, 1627350662, 1627793459, 1628236256, 1628679053, 1629121850, 1629564647, 1630007445, 1630450242, 1630893039, 1631335836, 1631778633, 1632221430, 1632664227, 1633107024, 1633549821, 1633992618, 1634435415, 1634878212, 1635321009, 1635763806, 1636206603, 1636649400, 1637092198, 1637534995, 1637977792, 1638420589, 1638863386, 1639306183, 1639748980, 1640191777, 1640634574, 1641077371, 1641520168, 1641962965, 1642405762, 1642848559, 1643291356, 1643734153, 1644176951, 1644619748, 1645062545, 1645505342, 1645948139, 1646390936, 1646833733, 1647276530, 1647719327, 1648162124, 1648604921, 1649047718, 1649490515, 1649933312, 1650376109, 1650818906, 1651261704, 1651704501, 1652147298, 1652590095, 1653032892, 1653475689, 1653918486, 1654361283, 1654804080, 1655246877, 1655689674, 1656132471, 1656575268, 1657018065, 1657460862, 1657903660, 1658346457, 1658789254, 1659232051, 1659674848, 1660117645, 1660560442, 1661003239, 1661446036, 1661888833, 1662331630, 1662774427, 1663217224, 1663660021, 1664102818, 1664545615, 1664988413, 1665431210, 1665874007, 1666316804, 1666759601, 1667202398, 1667645195, 1668087992, 1668530789, 1668973586, 1669416383, 1669859180, 1670301977, 1670744774, 1671187571, 1671630368, 1672073166, 1672515963, 1672958760, 1673401557, 1673844354, 1674287151, 1674729948, 1675172745, 1675615542, 1676058339, 1676501136, 1676943933, 1677386730, 1677829527, 1678272324, 1678715121, 1679157919, 1679600716, 1680043513, 1680486310, 1680929107, 1681371904, 1681814701, 1682257498, 1682700295, 1683143092, 1683585889, 1684028686, 1684471483, 1684914280, 1685357077, 1685799875, 1686242672, 1686685469, 1687128266, 1687571063, 1688013860, 1688456657, 1688899454, 1689342251, 1689785048, 1690227845, 1690670642, 1691113439, 1691556236, 1691999033, 1692441830, 1692884628, 1693327425, 1693770222, 1694213019, 1694655816, 1695098613, 1695541410, 1695984207, 1696427004, 1696869801, 1697312598, 1697755395, 1698198192, 1698640989, 1699083786, 1699526583, 1699969381, 1700412178, 1700854975, 1701297772, 1701740569, 1702183366, 1702626163, 1703068960, 1703511757, 1703954554, 1704397351, 1704840148, 1705282945, 1705725742, 1706168539, 1706611336, 1707054134, 1707496931, 1707939728, 1708382525, 1708825322, 1709268119, 1709710916, 1710153713, 1710596510, 1711039307, 1711482104, 1711924901],
        "returns_pct": [0.0, 2.89, 6.46, 7.34, 9.04, 9.15, 10.99, 12.89, 11.35, 13.21, 13.72, 14.53, 15.06, 13.19, 16.64, 17.92, 18.0, 17.87, 20.73, 22.19, 22.91, 23.96, 25.84, 25.86, 26.76, 23.63, 18.4, 20.44, 17.87, 16.27, 13.11, 20.09, 21.74, 23.61, 24.8, 24.2, 26.71, 26.06, 27.29, 26.05, 28.49, 28.65, 25.38, 25.06, 22.2, 25.53, 22.04, 25.33, 24.05, 27.34, 29.73, 30.57, 28.31, 27.91, 29.06, 28.47, 31.81, 32.73, 35.42, 36.12, 40.84, 42.52, 43.36, 44.94, 43.37, 45.51, 43.77, 45.71, 48.62, 51.1, 52.82, 54.95, 57.14, 60.47, 62.15, 63.89, 64.01, 65.08, 70.08, 69.58, 73.39, 75.77, 65.76, 52.67, 56.02, 43.54, 23.7, 20.24, 35.43, 35.43, 29.0, 37.39, 59.76, 56.6, 61.67, 54.54, 58.56, 62.87, 62.94, 65.53, 63.55, 67.03, 68.38, 73.2, 77.15, 80.89, 82.24, 94.19, 96.2, 98.58, 101.21, 104.47, 102.44, 104.0, 108.95, 108.25, 111.15, 112.33, 117.69, 120.54, 115.33, 109.91, 111.85, 105.52, 106.42, 107.62, 111.71, 124.88, 117.67, 113.7, 114.56, 102.79, 118.88, 116.4, 118.1, 115.86, 120.11, 124.58, 125.71, 123.01, 130.53, 129.96, 135.4, 133.96, 135.08, 134.31, 132.45, 138.61, 136.12, 137.5, 141.17, 143.39, 141.93, 137.97, 133.47, 130.63, 136.22, 141.5, 137.9, 138.01, 140.19, 144.16, 154.22, 155.25, 156.36, 158.93, 155.62, 151.03, 150.8, 151.03, 150.89, 153.75, 153.22, 154.52, 157.65, 159.43, 158.51, 163.53, 168.2, 172.04, 175.01, 171.82, 173.62, 178.63, 174.13, 175.82, 175.24, 176.55, 174.62, 177.46, 182.56, 183.75, 180.44, 180.98, 173.1, 178.07, 171.63, 170.3, 170.14, 178.49, 185.03, 189.85, 196.02, 205.73, 196.97, 201.0, 208.96, 199.89, 198.9, 197.13, 205.33, 197.28, 198.55, 211.34, 207.95, 204.76, 207.26, 204.31, 188.05, 181.65, 197.47, 196.46, 190.83, 195.47, 187.11, 187.32, 186.85, 173.28, 170.22, 189.48, 198.72, 206.72, 200.71, 196.13, 193.86, 188.35, 181.77, 173.26, 184.51, 163.91, 162.23, 151.64, 154.35, 167.69, 165.09, 159.5, 142.12, 137.95, 151.75, 149.2, 148.96, 171.44, 170.28, 191.46, 193.17, 216.05, 230.25, 203.32, 223.79, 215.72, 214.84, 184.65, 162.31, 179.52, 169.33, 172.77, 154.28, 144.81, 141.97, 127.89, 116.58, 121.03, 134.08, 137.9, 121.1, 103.73, 119.94, 115.8, 116.55, 124.0, 116.3, 112.44, 99.44, 86.51, 79.24, 77.78, 68.05, 79.19, 89.88, 94.38, 101.26, 117.06, 132.66, 132.49, 145.26, 138.6, 127.69, 123.38, 122.65, 111.06, 122.01, 129.4, 127.04, 143.63, 136.64, 133.63, 137.1, 135.62, 137.88, 134.49, 138.35, 137.4, 144.5, 142.4, 152.26, 158.32, 159.42, 166.41, 170.59, 167.15, 167.6, 172.61, 170.98, 175.69, 180.64, 176.78, 180.46, 174.24, 167.91, 164.3, 159.17, 166.29, 170.44, 174.34, 166.76, 165.24, 165.98, 159.73, 156.35, 156.47, 165.03, 164.92, 158.33, 155.32, 147.92, 158.39, 166.93, 170.01, 176.77, 178.37, 178.47, 178.08, 185.73, 190.09, 190.37, 186.82, 185.25, 176.06, 191.13, 192.89, 219.09, 224.62, 226.89, 230.69, 230.46, 240.37, 236.43, 246.19, 257.82, 271.47, 276.59, 266.37, 264.63, 264.31, 264.39],
    },
}

fig = go.Figure()
for label, data in curves.items():
    dates = [datetime.fromtimestamp(t) for t in data['timestamps']]
    fig.add_trace(go.Scatter(
        x=dates, y=data['returns_pct'],
        mode='lines', name=label,
        line=dict(color=data['color'], width=2),
    ))

fig.update_layout(
    title="All Strategies — Cumulative Return %",
    xaxis_title="Date",
    yaxis_title="Cumulative Return %",
    template="plotly_white",
    hovermode="x unified",
)
fig.show()

**References:**

- Bailey, D. H. & López de Prado, M. (2012). "The Sharpe Ratio Efficient Frontier." *Journal of Risk*, 15(2), 3–44. — Probabilistic Sharpe Ratio (PSR): estimates P(true Sharpe > 0 | observed data), accounting for sample size and higher moments.
- Pik, J. et al. (2025). *Hands-On AI Trading*. Wiley. p. 426. — Lookahead bias warning for pre-trained HuggingFace models with backtests starting 2019.
- Ansari, A. F. et al. (2024). "Chronos: Learning the Language of Time Series." [arXiv:2403.07815](https://arxiv.org/abs/2403.07815). — Pre-training corpus may overlap with backtest period.

## Conclusion

### Slide 18 — Closing: What I Learned

What I Learned

- Ridge fails. A linear model on volatility features tracks the market with extra drawdown.
- CNN generates real alpha. The temporal split architecture captures genuine patterns — 0.093 alpha with 0.278 beta.
- Foundation models change the game. Zero training → Sharpe 0.727. Three fine-tuning steps → 0.846.
- Less effort, better results. Each generation requires less human engineering and produces better outcomes.
- But alpha ≠ returns. The CNN has the cleanest alpha. Chronos has the highest total returns. Know what you're optimizing for.

The foundation model era is here.

Thank you.

---

Three generations of machine learning. Ridge regression fails — negative alpha, market tracking with extra pain. The CNN generates genuine independent returns — real alpha, low beta. And the foundation model? It arrives with zero training and outperforms both.

The arc isn't just better numbers. It's less work. From manual feature engineering to weekly retraining to nothing at all. Each generation pushes more of the intelligence into the model itself and requires less from the human.

But I want to leave you with a nuance: alpha and returns are not the same thing. The CNN generates the purest independent signal. Chronos generates the highest total returns. Which one you prefer depends on what you're building — a market-neutral hedge fund or a long-biased portfolio.

The foundation model era is here. Thank you.


**References:**

- Pik, J. et al. (2025). *Hands-On AI Trading with Python, QuantConnect, and AWS*. Wiley. — All exercises, strategies, and backtest results.
- Hoerl, A. E. & Kennard, R. W. (1970). "Ridge Regression." *Technometrics*, 12(1), 55–82. — Classical ML baseline.
- LeCun, Y. et al. (1989). "Backpropagation Applied to Handwritten Zip Code Recognition." *Neural Computation*, 1(4), 541–551. — Deep learning foundation.
- Ansari, A. F. et al. (2024). "Chronos: Learning the Language of Time Series." [arXiv:2403.07815](https://arxiv.org/abs/2403.07815). — Foundation model era.

## Full Bibliography

**Primary Source**

- Pik, J. et al. (2025). *Hands-On AI Trading with Python, QuantConnect, and AWS*. Wiley.

**Academic Papers**

- Ansari, A. F. et al. (2024). "Chronos: Learning the Language of Time Series." [arXiv:2403.07815](https://arxiv.org/abs/2403.07815).
- Bailey, D. H. & López de Prado, M. (2012). "The Sharpe Ratio Efficient Frontier." *Journal of Risk*, 15(2), 3–44.
- Bommasani, R. et al. (2021). "On the Opportunities and Risks of Foundation Models." Stanford CRFM. [arXiv:2108.07258](https://arxiv.org/abs/2108.07258).
- Brown, T. et al. (2020). "Language Models are Few-Shot Learners." [arXiv:2005.14165](https://arxiv.org/abs/2005.14165).
- Hoerl, A. E. & Kennard, R. W. (1970). "Ridge Regression: Biased Estimation for Nonorthogonal Problems." *Technometrics*, 12(1), 55–67.
- Hoerl, A. E. & Kennard, R. W. (1970). "Ridge Regression: Applications to Nonorthogonal Problems." *Technometrics*, 12(1), 69–82.
- Jensen, M. C. (1968). "The Performance of Mutual Funds in the Period 1945–1964." *The Journal of Finance*, 23(2), 389–416.
- Kim, J. H. (2019). "Multicollinearity and misleading statistical results." *Korean Journal of Anesthesiology*, 72(6), 558–569. [doi:10.4097/kja.19087](https://pmc.ncbi.nlm.nih.gov/articles/PMC6900425/).
- LeCun, Y. et al. (1989). "Backpropagation Applied to Handwritten Zip Code Recognition." *Neural Computation*, 1(4), 541–551.
- Markowitz, H. (1952). "Portfolio Selection." *The Journal of Finance*, 7(1), 77–91.
- Raffel, C. et al. (2019). "Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer." [arXiv:1910.10683](https://arxiv.org/abs/1910.10683).
- Sharpe, W. F. (1966). "Mutual Fund Performance." *The Journal of Business*, 39(1), 119–138.
- Sharpe, W. F. (1994). "The Sharpe Ratio." *The Journal of Portfolio Management*, 21(1), 49–58.
- Vaswani, A. et al. (2017). "Attention Is All You Need." [arXiv:1706.03762](https://arxiv.org/abs/1706.03762).
- Wilson, C.-A. (2025). "Explainable AI in Finance." CFA Institute Research & Policy Center. [doi:10.56227/25.1.25](https://rpc.cfainstitute.org/research/reports/2025/explainable-ai-in-finance).

**Books**

- Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press.
- Murphy, J. J. (1999). *Technical Analysis of the Financial Markets*. New York Institute of Finance.
- Wilder, J. W. (1978). *New Concepts in Technical Trading Systems*. Trend Research.

---

*Generated on 2026-02-17 from presentation source materials.*

*For educational purposes only. Nothing in this notebook constitutes financial advice.*